In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader


In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LGaimers/해커톤/train/train.csv")

In [ ]:
# '영업장명'과 '메뉴명' 분리
df[['영업장명', '메뉴명']] = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)
df['영업일자'] = pd.to_datetime(df['영업일자'])  # 문자열 → datetime 변환

In [ ]:
print(df['영업일자'].min(), df['영업일자'].max())

2023-01-01 00:00:00 2024-06-15 00:00:00


In [ ]:
zero_sales_dict = {}

# 메뉴별 0판매 구간 탐지
for menu, group in df.groupby('영업장명_메뉴명'):
    group = group.sort_values('영업일자').reset_index(drop=True)
    zero_sales = group[group['매출수량'] == 0].copy()

    if zero_sales.empty:
        continue

    zero_sales['날짜차이'] = zero_sales['영업일자'].diff().dt.days.fillna(1)
    group_id = (zero_sales['날짜차이'] != 1).cumsum()

    result_list = []
    for _, sub in zero_sales.groupby(group_id):
        dates = sub['영업일자'].dt.date.tolist()
        if len(dates) == 1:
            result_list.append(f"({dates[0]})")
        else:
            result_list.append(f"({dates[0]}, {dates[-1]})")
    zero_sales_dict[menu] = result_list

is_zero_sales_period가 1이면 일주일 이상 판매 중단 기간

In [ ]:
n = 7  # 판매 중단으로 간주할 최소 연속일 수

df['is_zero_sales_period'] = 0  # 초기화

for menu, periods in zero_sales_dict.items():
    for period in periods:
        dates = period.strip("()").split(", ")
        if len(dates) == 1:
            continue  # 단일 날짜는 제외
        else:
            d1 = pd.to_datetime(dates[0])
            d2 = pd.to_datetime(dates[1])
            duration = (d2 - d1).days + 1

            if duration >= n:
                df.loc[
                    (df['영업장명_메뉴명'] == menu) &
                    (df['영업일자'].between(d1, d2)),
                    'is_zero_sales_period'
                ] = 1

영업장별 4일 이상 매출 없는 기간 정기적 휴무일 지정

In [ ]:
import pandas as pd
import numpy as np

def find_store_zero_runs(df: pd.DataFrame,
                         date_col: str = '영업일자',
                         store_col: str = '영업장명',
                         sales_col: str = '매출수량',
                         min_run: int = 7,
                         treat_missing_as_zero: bool = True) -> pd.DataFrame:
    """
    영업장별로 (모든 메뉴 합계 기준) 매출수량==0 이 min_run일 이상 연속인 기간을 반환.
    반환 컬럼: [영업장명, start, end, run_length]

    treat_missing_as_zero:
      True  -> 원본에 없는 날짜(결측 일자)를 0으로 간주(리샘플링 후 0 채움)
      False -> 원본에 있는 날짜만 사용(연속성은 원본 일자 기준)
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])

    # 1) 영업장×날짜별 총 매출(모든 메뉴 합계)
    daily = (df.groupby([store_col, date_col], as_index=False)[sales_col]
               .sum()
               .rename(columns={sales_col: 'store_sales'}))

    results = []

    for store, g in daily.groupby(store_col, sort=False):
        g = g.sort_values(date_col)

        if treat_missing_as_zero:
            # 영업장별 전체 날짜 인덱스로 리샘플링 → 결측일 0으로 채움
            full_idx = pd.date_range(g[date_col].min(), g[date_col].max(), freq='D')
            s = (g.set_index(date_col)['store_sales']
                   .reindex(full_idx)
                   .fillna(0.0))
            s.index.name = date_col
        else:
            # 원본에 존재하는 날짜만 사용(날짜 간격이 띄엄띄엄일 수 있음)
            s = g.set_index(date_col)['store_sales'].copy()

        # 2) 0/비0 플래그
        z = (s == 0).astype(int)

        if z.sum() == 0:
            continue  # 0이 한 번도 없으면 스킵

        # 3) 연속 구간(run) 구분: 값이 바뀔 때마다 run_id 증가
        run_id = (z.diff().fillna(z.iloc[0]) != 0).cumsum()

        # 4) run별 요약
        tmp = (
            pd.DataFrame({'flag_zero': z, 'run_id': run_id})
            .groupby('run_id', group_keys=False)
            .apply(lambda d: pd.Series({
                'flag_zero': d['flag_zero'].iloc[0],
                'start':     d.index.min(),
                'end':       d.index.max(),
                'run_length': len(d)
            }))
        )

        # 5) 0인 구간 + 길이 필터
        zero_runs = tmp[(tmp['flag_zero'] == 1) & (tmp['run_length'] >= min_run)]
        if zero_runs.empty:
            continue

        # 6) 결과 정리
        for _, row in zero_runs.iterrows():
            results.append({
                store_col: store,
                'start':   row['start'],
                'end':     row['end'],
                'run_length': int(row['run_length']),
            })

    out = pd.DataFrame(results).sort_values([store_col, 'start']).reset_index(drop=True)
    return out


In [ ]:
def print_zero_runs_by_store(zero_runs_df, store_col='영업장명'):
    """
    find_store_zero_runs() 결과를 영업장별로 보기 좋게 출력하는 함수.
    """
    stores = zero_runs_df[store_col].unique()
    for store in stores:
        print(f"\n=== {store} ===")
        runs = zero_runs_df[zero_runs_df[store_col] == store]
        if runs.empty:
            print("  (연속 0 구간 없음)")
        else:
            for _, row in runs.iterrows():
                start_str = row['start'].strftime('%Y-%m-%d')
                end_str = row['end'].strftime('%Y-%m-%d')
                print(f"  {start_str} ~ {end_str} ({row['run_length']}일)")


In [ ]:
zero_periods = find_store_zero_runs(
    df,  # 원본 데이터프레임
    date_col='영업일자',
    store_col='영업장명',
    sales_col='매출수량',
    min_run=4,
    treat_missing_as_zero=True
)

print_zero_runs_by_store(zero_periods)



=== 느티나무 셀프BBQ ===
  2023-03-01 ~ 2023-03-09 (9일)
  2024-03-03 ~ 2024-03-13 (11일)

=== 담하 ===
  2023-03-02 ~ 2023-03-09 (8일)
  2024-03-04 ~ 2024-03-12 (9일)

=== 라그로타 ===
  2023-03-02 ~ 2023-03-09 (8일)
  2024-03-04 ~ 2024-03-12 (9일)

=== 미라시아 ===
  2023-03-02 ~ 2023-03-09 (8일)
  2024-03-04 ~ 2024-03-12 (9일)

=== 연회장 ===
  2023-03-01 ~ 2023-03-09 (9일)
  2024-03-04 ~ 2024-03-13 (10일)

=== 카페테리아 ===
  2023-03-02 ~ 2023-03-09 (8일)
  2024-03-04 ~ 2024-03-13 (10일)

=== 포레스트릿 ===
  2023-03-02 ~ 2023-03-11 (10일)
  2023-03-13 ~ 2023-03-31 (19일)
  2023-04-03 ~ 2023-04-06 (4일)
  2023-04-10 ~ 2023-04-13 (4일)
  2023-04-17 ~ 2023-04-20 (4일)
  2023-08-28 ~ 2023-09-01 (5일)
  2023-09-04 ~ 2023-09-07 (4일)
  2023-09-11 ~ 2023-09-15 (5일)
  2023-09-18 ~ 2023-09-22 (5일)
  2023-11-27 ~ 2023-12-05 (9일)
  2024-03-04 ~ 2024-03-26 (23일)
  2024-04-29 ~ 2024-05-02 (4일)
  2024-05-20 ~ 2024-05-23 (4일)
  2024-05-27 ~ 2024-05-31 (5일)
  2024-06-10 ~ 2024-06-13 (4일)

=== 화담숲주막 ===
  2023-01-01 ~ 2023-03-30 (89일)
  2023-

/tmp/ipython-input-3391356828.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({
/tmp/ipython-input-3391356828.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({
/tmp/ipython-input-3391356828.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the group

In [ ]:
import pandas as pd
import numpy as np

# --- 1) 영업장별 "연속 0(>=min_run일)" 구간 캘린더 생성 ---
def _find_store_zero_runs_calendar(df: pd.DataFrame,
                                   date_col='영업일자',
                                   store_col='영업장명',
                                   sales_col='매출수량',
                                   min_run=7,
                                   treat_missing_as_zero=True) -> pd.DataFrame:
    """
    반환: [영업장명, ds, 휴무일] (연속 0 구간의 모든 날짜에 휴무일=1)
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])

    # 영업장×날짜 총매출(모든 메뉴 합)
    daily = (df.groupby([store_col, date_col], as_index=False)[sales_col]
               .sum()
               .rename(columns={sales_col: 'store_sales'}))

    rows = []
    for store, g in daily.groupby(store_col, sort=False):
        g = g.sort_values(date_col)

        # 일 단위 인덱스 확보
        if treat_missing_as_zero:
            full_idx = pd.date_range(g[date_col].min(), g[date_col].max(), freq='D')
            s = (g.set_index(date_col)['store_sales']
                   .reindex(full_idx)
                   .fillna(0.0))
            s.index.name = date_col
        else:
            s = g.set_index(date_col)['store_sales'].copy()

        z = (s == 0).astype(int)
        if z.sum() == 0:
            continue

        # run 분할
        run_id = (z.diff().fillna(z.iloc[0]) != 0).cumsum()
        tmp = (
            pd.DataFrame({'flag_zero': z, 'run_id': run_id})
            .groupby('run_id', group_keys=False)
            .apply(lambda d: pd.Series({
                'flag_zero': d['flag_zero'].iloc[0],
                'start':     d.index.min(),
                'end':       d.index.max(),
                'length':    len(d)
            }))
        )

        # 0구간 + 길이 조건
        tmp = tmp[(tmp['flag_zero'] == 1) & (tmp['length'] >= min_run)]
        if tmp.empty:
            continue

        # 날짜 캘린더로 확장
        for _, r in tmp.iterrows():
            days = pd.date_range(r['start'], r['end'], freq='D')
            rows.append(pd.DataFrame({
                store_col: store,
                'ds': days,
                '휴무일': 1
            }))

    if rows:
        off_cal = pd.concat(rows, ignore_index=True).drop_duplicates([store_col, 'ds'])
    else:
        off_cal = pd.DataFrame(columns=[store_col, 'ds', '휴무일'])
    return off_cal


# --- 2) 기존 휴무일 삭제 후, 새 규칙으로 생성 ---
def rebuild_offdays(df: pd.DataFrame,
                    date_col='영업일자',
                    store_col='영업장명',
                    menu_col='메뉴명',
                    sales_col='매출수량',
                    min_run=7,
                    treat_missing_as_zero=True,
                    ) -> pd.DataFrame:
    """
    기존 df의 휴무일 컬럼을 제거하고, 연속0(>=min_run) + 특정 매장 월요일 규칙으로 새로 생성.
    반환: 휴무일(0/1)이 새로 생성된 df
    """
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col])

    # 0) 기존 휴무일 컬럼 제거
    if '휴무일' in out.columns:
        out = out.drop(columns=['휴무일'])

    # 1) 연속 0 구간 캘린더 생성
    off_cal = _find_store_zero_runs_calendar(out, date_col, store_col, sales_col,
                                             min_run=min_run,
                                             treat_missing_as_zero=treat_missing_as_zero)

    # 2) 기본 0으로 초기화 후, 캘린더 병합으로 1 세팅
    out['휴무일'] = 0
    if not off_cal.empty:
        off_cal = off_cal.copy()
        off_cal['ds'] = pd.to_datetime(off_cal['ds'])
        out = out.merge(off_cal[[store_col, 'ds', '휴무일']].rename(columns={'휴무일':'휴무일_from_zero'}),
                        how='left', left_on=[store_col, date_col], right_on=[store_col, 'ds'])
        out['휴무일'] = out['휴무일_from_zero'].fillna(0).astype(int)
        out.drop(columns=['ds','휴무일_from_zero'], inplace=True)

    # 4) 최종 정리: int8로 다운캐스팅(선택)
    out['휴무일'] = out['휴무일'].astype('int8')
    return out


# === 실행 예시 ===
# df = ... (원본)
df = rebuild_offdays(
    df,
    date_col='영업일자',
    store_col='영업장명',
    menu_col='메뉴명',
    sales_col='매출수량',
    min_run=4,
    treat_missing_as_zero=True,   # 결측 일자도 0으로 간주하여 보수적으로 탐지
)

/tmp/ipython-input-303728899.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({
/tmp/ipython-input-303728899.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({
/tmp/ipython-input-303728899.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping

In [ ]:
def count_weekdays_for_non_offday_zero_sales(df, date_col='영업일자', store_col='영업장명',
                                             sales_col='매출수량', offday_col='휴무일'):
    """
    휴무일이 아닌데 매출 0인 날짜의 요일 분포를 카운트
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])

    # 1) 영업장 × 날짜 총매출
    daily_sales = (
        df.groupby([store_col, date_col], as_index=False)[sales_col]
          .sum()
          .rename(columns={sales_col: 'store_sales'})
    )

    # 2) 휴무일 정보 병합
    if offday_col in df.columns:
        daily_sales = daily_sales.merge(
            df[[store_col, date_col, offday_col]].drop_duplicates(),
            on=[store_col, date_col],
            how='left'
        )
    else:
        daily_sales[offday_col] = 0

    # 3) 조건 필터
    target = daily_sales[
        (daily_sales[offday_col] == 0) &
        (daily_sales['store_sales'] == 0)
    ].copy()

    # 4) 요일 컬럼 추가 (0=월, 6=일)
    target['weekday'] = target[date_col].dt.weekday

    # 5) 요일 이름 매핑
    weekday_map = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
    target['weekday_name'] = target['weekday'].map(weekday_map)

    # 6) 요일 카운트
    weekday_counts = target['weekday_name'].value_counts().reindex(weekday_map.values(), fill_value=0)

    return target, weekday_counts


# === 실행 예시 ===
target_df, weekday_counts = count_weekdays_for_non_offday_zero_sales(df)

print("📊 요일별 매출 0(휴무일 아님) 카운트")
print(weekday_counts)

# 필요하면 영업장별 상세 출력
for store, group in target_df.groupby('영업장명'):
    print(f"\n=== {store} ===")
    print(group['weekday_name'].value_counts().sort_index())


📊 요일별 매출 0(휴무일 아님) 카운트
weekday_name
월    159
화     12
수      6
목      9
금      3
토      0
일     26
Name: count, dtype: int64

=== 느티나무 셀프BBQ ===
weekday_name
월    15
화     2
Name: count, dtype: int64

=== 라그로타 ===
weekday_name
월    51
화     2
Name: count, dtype: int64

=== 연회장 ===
weekday_name
수     2
월     4
일    25
화     2
Name: count, dtype: int64

=== 포레스트릿 ===
weekday_name
금     3
목     7
수     4
월    21
일     1
화     6
Name: count, dtype: int64

=== 화담숲주막 ===
weekday_name
목     1
월    34
Name: count, dtype: int64

=== 화담숲카페 ===
weekday_name
목     1
월    34
Name: count, dtype: int64


In [ ]:
import pandas as pd

def count_zero_sales_by_store_month_weekday_excluding_offdays(
    df: pd.DataFrame,
    date_col: str = '영업일자',
    store_col: str = '영업장명',
    sales_col: str = '매출수량',
    offday_col: str = '휴무일',
    month_format: str = '%Y-%m'
):
    """
    휴무일 제외 후, 업장별·월별 '매출수량=0'인 날짜의 요일 카운트를 반환합니다.
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])

    # 1) 영업장×날짜 총매출 (모든 메뉴 합)
    daily_sales = (
        df.groupby([store_col, date_col], as_index=False)[sales_col]
          .sum()
          .rename(columns={sales_col: 'store_sales'})
    )

    # 2) 휴무일 정보 병합 & 제외
    if offday_col in df.columns:
        offday_info = (
            df[[store_col, date_col, offday_col]]
            .drop_duplicates([store_col, date_col])
        )
        daily_sales = daily_sales.merge(offday_info, on=[store_col, date_col], how='left')
        daily_sales = daily_sales[daily_sales[offday_col].fillna(0).astype(int) == 0]

    # 3) 0매출 날짜만 필터
    zero_sales = daily_sales[daily_sales['store_sales'] == 0].copy()

    # 4) 요일 / 월 파생
    zero_sales['weekday'] = zero_sales[date_col].dt.weekday  # 월=0 … 일=6
    weekday_map = {0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'}
    zero_sales['weekday_name'] = zero_sales['weekday'].map(weekday_map)
    zero_sales['month'] = zero_sales[date_col].dt.strftime(month_format)

    # 5) Long 형태 집계
    counts_long = (
        zero_sales.groupby([store_col, 'month', 'weekday_name'], as_index=False)
                  .size()
                  .rename(columns={'size':'zero_days'})
    )

    # 6) Pivot 형태
    counts_pivot = (
        counts_long.pivot_table(index=[store_col, 'month'],
                                columns='weekday_name',
                                values='zero_days',
                                aggfunc='sum',
                                fill_value=0)
                    .reindex(columns=['월','화','수','목','금','토','일'], fill_value=0)
                    .sort_index()
    )

    return counts_pivot, counts_long


# 사용 예시
pivot_all, long_all = count_zero_sales_by_store_month_weekday_excluding_offdays(df)

print("=== 업장별·월별 0매출 요일 카운트 (휴무일 제외) ===")
display(pivot_all)

=== 업장별·월별 0매출 요일 카운트 (휴무일 제외) ===


weekday_name        월  화  수  목  금  토  일
영업장명       month                       
느티나무 셀프BBQ 2023-03  1  1  0  0  0  0  0
           2023-04  1  0  0  0  0  0  0
           2023-07  2  0  0  0  0  0  0
           2023-09  4  0  0  0  0  0  0
           2023-10  1  0  0  0  0  0  0
...                .. .. .. .. .. .. ..
화담숲카페      2023-10  1  0  0  0  0  0  0
           2023-11  1  0  0  0  0  0  0
           2024-04  5  0  0  0  0  0  0
           2024-05  3  0  0  0  0  0  0
           2024-06  2  0  0  0  0  0  0

[74 rows x 7 columns]

In [ ]:
def assign_offdays_by_zero_sales(pivot_table, threshold=3):
    """
    pivot_table: count_zero_sales_by_store_month_weekday_excluding_offdays()의 pivot_all 결과
    threshold: 해당 요일이 이 횟수 이상이면 휴무일로 지정
    """
    offday_records = []

    for (store, month), row in pivot_table.iterrows():
        # 3회 이상인 요일 추출
        offdays = [day for day, cnt in row.items() if cnt >= threshold]
        if offdays:
            offday_records.append({
                '영업장명': store,
                '월': month,
                '휴무요일': offdays
            })

    return pd.DataFrame(offday_records)


# 사용 예시
pivot_all, _ = count_zero_sales_by_store_month_weekday_excluding_offdays(df)
offday_plan_df = assign_offdays_by_zero_sales(pivot_all, threshold=3)

print("=== 휴무일 지정 계획 ===")
for _, row in offday_plan_df.iterrows():
    print(f"{row['영업장명']} ({row['월']}): {', '.join(row['휴무요일'])}")


=== 휴무일 지정 계획 ===
느티나무 셀프BBQ (2023-09): 월
라그로타 (2023-02): 월
라그로타 (2023-03): 월
라그로타 (2023-04): 월
라그로타 (2023-05): 월
라그로타 (2023-06): 월
라그로타 (2023-07): 월
라그로타 (2023-08): 월
라그로타 (2023-09): 월
라그로타 (2023-11): 월
라그로타 (2024-04): 월
라그로타 (2024-05): 월
연회장 (2023-03): 일
연회장 (2023-04): 일
포레스트릿 (2023-05): 월
포레스트릿 (2023-10): 월
포레스트릿 (2023-11): 월
포레스트릿 (2024-04): 월
화담숲주막 (2023-04): 월
화담숲주막 (2023-05): 월
화담숲주막 (2023-06): 월
화담숲주막 (2023-07): 월
화담숲주막 (2023-08): 월
화담숲주막 (2023-09): 월
화담숲주막 (2024-04): 월
화담숲주막 (2024-05): 월
화담숲카페 (2023-04): 월
화담숲카페 (2023-05): 월
화담숲카페 (2023-06): 월
화담숲카페 (2023-07): 월
화담숲카페 (2023-08): 월
화담숲카페 (2023-09): 월
화담숲카페 (2024-04): 월
화담숲카페 (2024-05): 월


In [ ]:
import pandas as pd
import numpy as np

def add_custom_offday_only(
    df: pd.DataFrame,
    date_col: str = '영업일자',
    store_col: str = '영업장명',
    offday_col: str = '휴무일',      # 0=영업, 1=휴무
    out_col: str = 'custom_offday'
) -> pd.DataFrame:
    """
    기존 휴무일(offday_col)==0(영업일)에서만 규칙을 적용해 custom_offday=1을 부여합니다.
    규칙:
      - 포레스트릿: 4,5,9,10,11월 & 월요일(weekday=0)
      - 연회장: 일요일(weekday=6)
      - 화담숲주막/화담숲카페/라그로타/느티나무 셀프BBQ: 월요일(weekday=0)
    """
    if offday_col not in df.columns:
        raise KeyError(f"'{offday_col}' 컬럼이 필요합니다. (0=영업, 1=휴무)")

    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col])

    # --- 휴무일 컬럼 정규화 (bool/문자열/숫자 모두 처리) ---
    off = out[offday_col]
    if off.dtype == bool or set(off.dropna().unique()) <= {True, False}:
        off = off.astype(int)
    elif off.dtype == 'O':  # 'holiday'/'non-holiday' 등 문자열
        off = (off.replace({'holiday': 1, 'non-holiday': 0})
                 .pipe(pd.to_numeric, errors='coerce')
                 .fillna(0).astype(int))
    else:
        off = pd.to_numeric(off, errors='coerce').fillna(0).astype(int)
    # 영업 중(휴무 아님) 조건
    is_op = (off == 0).values

    # --- 파생 ---
    out['weekday'] = out[date_col].dt.weekday  # 월=0 … 일=6
    out['month']   = out[date_col].dt.month

    # --- 기본 0 ---
    custom = np.zeros(len(out), dtype=np.int8)

    # 1) 포레스트릿: 4,5,9,10,11월 & 월요일 & 영업 중
    mask = (
        (out[store_col] == '포레스트릿') &
        (out['month'].isin([4,5,9,10,11])) &
        (out['weekday'] == 0) & is_op
    )
    custom = np.where(mask, 1, custom)

    # 2) 연회장: 일요일 & 영업 중
    mask = (
        (out[store_col] == '연회장') &
        (out['weekday'] == 6) & is_op
    )
    custom = np.where(mask, 1, custom)

    # 3) 화담숲주막: 월요일 & 영업 중
    mask = (
        (out[store_col] == '화담숲주막') &
        (out['weekday'] == 0) & is_op
    )
    custom = np.where(mask, 1, custom)

    # 4) 화담숲카페: 월요일 & 영업 중
    mask = (
        (out[store_col] == '화담숲카페') &
        (out['weekday'] == 0) & is_op
    )
    custom = np.where(mask, 1, custom)

    # 5) 라그로타: 월요일 & 영업 중
    mask = (
        (out[store_col] == '라그로타') &
        (out['weekday'] == 0) & is_op
    )
    custom = np.where(mask, 1, custom)

    # 6) 느티나무 셀프BBQ: 월요일 & 영업 중
    mask = (
        (out[store_col] == '느티나무 셀프BBQ') &
        (out['weekday'] == 0) & is_op
    )
    custom = np.where(mask, 1, custom)

    out[out_col] = custom.astype('int8')

    # 정리
    out.drop(columns=['weekday','month'], inplace=True)
    return out


In [ ]:
df = add_custom_offday_only(df, date_col='영업일자', store_col='영업장명', offday_col='휴무일', out_col='custom_offday')

요일별 0매출 비율이 90% 이상이거나 판매 중지 기간일 시에 이상치 보간에서 제외

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import STL, seasonal_decompose

# ─────────────────────────────────────────────────────────────
# 요일별 0매출 비율로 '정기 휴무 요일' 자동 판정
# ─────────────────────────────────────────────────────────────
def _detect_weekly_off_days_from_series(ts: pd.Series, min_weeks=8, tau=0.8):
    tmp = pd.DataFrame({"y": ts})
    tmp["weekday"] = tmp.index.weekday
    weekly_off = set()
    for w in range(7):
        sub = tmp.loc[tmp["weekday"] == w, "y"]
        n_obs = sub.notna().sum()
        if n_obs < min_weeks:
            continue
        zero_rate = (sub.fillna(0) == 0).mean()
        if zero_rate >= tau:
            weekly_off.add(w)
    return weekly_off

# ─────────────────────────────────────────────────────────────
# 단일 (영업장명, 메뉴명) 그룹 청소 + 고립 0/이상치 선형보간
# ─────────────────────────────────────────────────────────────
def _clean_one_group(subset: pd.DataFrame,
                     period=7, max_interp_gap=None,  # None이면 제한 없이 보간
                     min_weeks=8, tau=0.8,
                     offday_col='휴무일', custom_off_col='custom_offday') -> pd.Series | None:
    """
    subset: 단일 (영업장명, 메뉴명) 데이터프레임
            반드시 포함: ['영업일자','매출수량','is_zero_sales_period']
            선택 포함: ['휴무일','custom_offday'] (없으면 0으로 간주)
    return: 청소 완료된 시계열 (DatetimeIndex, float)
    """
    # 1) 일자 집계 & 정렬
    g = subset.groupby('영업일자').agg(
        매출수량=('매출수량', 'sum'),
        is_zero=('is_zero_sales_period', 'max'),
        휴무일=(offday_col, 'max') if offday_col in subset.columns else ('영업일자', lambda x: 0),
        custom_offday=(custom_off_col, 'max') if custom_off_col in subset.columns else ('영업일자', lambda x: 0),
    ).sort_index()

    # 2) 전체 일자 인덱스로 재색인
    full_idx = pd.date_range(g.index.min(), g.index.max(), freq='D')
    g = g.reindex(full_idx)

    ts = g['매출수량']
    structural_zero = g['is_zero'].fillna(0).astype(int).eq(1)
    offday_mask = (g['휴무일'].fillna(0).astype(int).eq(1)) | (g['custom_offday'].fillna(0).astype(int).eq(1))

    if len(ts) < period * 2:
        return None  # 데이터 부족

    # 3) 정기 휴무 요일 탐지 (판매 기록 기반)
    weekly_off_days = _detect_weekly_off_days_from_series(ts, min_weeks=min_weeks, tau=tau)
    weekday = pd.Series(ts.index.weekday, index=ts.index)
    weekly_off_mask = weekday.isin(weekly_off_days) & (ts.isna() | ts.eq(0))

    # 4) 하드 0(보간 금지) 정의
    hard_zero_mask = structural_zero | weekly_off_mask | offday_mask

    # 5) STL 분해용 시계열: 하드0는 NaN으로 두고, 짧은 결측만 제한 보간
    ts_for_decomp = ts.copy()
    ts_for_decomp[hard_zero_mask] = np.nan
    ts_for_decomp = ts_for_decomp.interpolate(
        method='linear',
        limit=max_interp_gap,  # None이면 제한 없음
        limit_direction='both'
    )

    # 6) 분해(STL→fallback)
    try:
        stl = STL(ts_for_decomp.dropna(), period=period, robust=True)
        resid = stl.fit().resid.reindex(ts_for_decomp.index)
    except Exception:
        decomp = seasonal_decompose(ts_for_decomp.dropna(), model='additive', period=period)
        resid = decomp.resid.reindex(ts_for_decomp.index)

    # 7) 이상치 마스크(3σ 초과), 하드0는 제외
    sigma = resid.std(skipna=True)
    outlier_mask = pd.Series(False, index=ts.index)
    if pd.notna(sigma) and sigma > 0:
        outlier_mask = resid.abs() > (3 * sigma)
        outlier_mask = outlier_mask.fillna(False) & (~hard_zero_mask)

    # 8) '고립된 0매출' 마스크: 정기/장기/휴무가 아닌데 값이 0
    isolated_zero_mask = ts.eq(0) & (~hard_zero_mask)

    # 9) 보간 대상: 고립된 0 ∪ 이상치
    target_mask = (isolated_zero_mask | outlier_mask)

    # 10) 선형보간 수행
    ts_clean = ts.copy()
    guard = ts_clean[hard_zero_mask].copy()           # 하드0 값 보존(대개 0 또는 NaN)
    ts_clean[target_mask] = np.nan                    # 보간 대상만 NaN으로
    ts_final = ts_clean.interpolate(
        method='linear',
        limit=max_interp_gap,                         # None이면 전부 선형보간
        limit_direction='both'
    )
    # 끝단 등 남은 결측 보완 (선형보간 불가 구간)
    if ts_final.isna().any():
        ts_final = ts_final.fillna(method='ffill').fillna(method='bfill')

    # 11) 하드0 복원(휴무 결측은 0 고정)
    ts_final[hard_zero_mask] = guard.fillna(0)

    return ts_final

# ─────────────────────────────────────────────────────────────
# 전체 df에 대해 그룹별 청소 수행
# ─────────────────────────────────────────────────────────────
def clean_all_series(df: pd.DataFrame,
                     store_col="영업장명", menu_col="메뉴명",
                     date_col="영업일자", value_col="매출수량",
                     zero_flag_col="is_zero_sales_period",
                     offday_col="휴무일", custom_off_col="custom_offday",
                     period=7, max_interp_gap=None, min_weeks=8, tau=0.8):
    """
    return:
      cleaned_series_dict: { "영업장_메뉴": pd.Series(DatetimeIndex) }
      failed_keys: 리스트 (에러/데이터부족 등으로 스킵된 키)
    """
    cleaned_series_dict = {}
    failed_keys = []

    cols = [store_col, menu_col, date_col, value_col, zero_flag_col]
    if offday_col in df.columns:
        cols.append(offday_col)
    if custom_off_col in df.columns:
        cols.append(custom_off_col)

    d = df[cols].copy()
    d[date_col] = pd.to_datetime(d[date_col])

    for (store, menu), subset in d.groupby([store_col, menu_col], sort=False):
        key = f"{store}_{menu}"
        try:
            # 열 이름 통일
            renamer = {
                date_col: '영업일자',
                value_col: '매출수량',
                zero_flag_col: 'is_zero_sales_period'
            }
            if offday_col in subset.columns:
                renamer[offday_col] = '휴무일'
            if custom_off_col in subset.columns:
                renamer[custom_off_col] = 'custom_offday'

            sub = subset.rename(columns=renamer)

            ts_clean = _clean_one_group(
                sub,
                period=period,
                max_interp_gap=max_interp_gap,
                min_weeks=min_weeks,
                tau=tau,
                offday_col='휴무일',
                custom_off_col='custom_offday'
            )
            if ts_clean is None:
                failed_keys.append((key, "데이터 부족"))
                continue
            cleaned_series_dict[key] = ts_clean
        except Exception as e:
            failed_keys.append((key, str(e)))
    return cleaned_series_dict, failed_keys


In [ ]:
# 1) 청소 실행
cleaned_series_dict, failed_keys = clean_all_series(
    df,                           # 원본 데이터프레임
    store_col="영업장명",
    menu_col="메뉴명",
    date_col="영업일자",
    value_col="매출수량",
    zero_flag_col="is_zero_sales_period",
    offday_col="휴무일",
    custom_off_col="custom_offday",
    period=7,
    max_interp_gap=5,          # 제한 없이 선형보간
    min_weeks=8,
    tau=0.85
)
print("스킵:", failed_keys[:5])

# 2) 병합 함수
def merge_cleaned_series(cleaned_dict: dict) -> pd.DataFrame:
    rows = []
    for key, series in cleaned_dict.items():
        store, menu = key.split("_", 1)
        df_tmp = pd.DataFrame({
            "영업장명": store,
            "메뉴명": menu,
            "영업일자": series.index,
            "매출수량": series.values
        })
        rows.append(df_tmp)
    return pd.concat(rows, ignore_index=True)

# 3) 병합 + 후처리
df1 = merge_cleaned_series(cleaned_series_dict)
df1['매출수량'] = df1['매출수량'].clip(lower=0)  # 음수 방지


/tmp/ipython-input-1713288838.py:103: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ts_final = ts_final.fillna(method='ffill').fillna(method='bfill')
/tmp/ipython-input-1713288838.py:103: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ts_final = ts_final.fillna(method='ffill').fillna(method='bfill')
/tmp/ipython-input-1713288838.py:103: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ts_final = ts_final.fillna(method='ffill').fillna(method='bfill')


스킵: []


In [ ]:
import pandas as pd

def add_off_and_zero_flags(df1: pd.DataFrame,
                           df: pd.DataFrame,
                           date_col='영업일자',
                           shop_col='영업장명',
                           menu_col='메뉴명',
                           combined_col='영업장명_메뉴명',
                           cols_to_add=('휴무일','custom_offday','is_zero_sales_period'),
                           prefer_df1=True):
    """
    df의 ['휴무일','custom_offday','is_zero_sales_period']를 df1에 머지하여 추가/보완합니다.
    - 키: (영업장명, 메뉴명, 영업일자)
    - df1에 기존 값이 있으면 보존하고, 결측만 df 값으로 채웁니다 (prefer_df1=True).
      False로 주면 df의 값을 우선 적용합니다.
    """
    df1 = df1.copy()
    df  = df.copy()

    # 0) 날짜 컬럼 형 변환
    df1[date_col] = pd.to_datetime(df1[date_col])
    df[date_col]  = pd.to_datetime(df[date_col])

    # 1) (영업장명, 메뉴명) 확보
    def ensure_shop_menu_columns(d):
        if shop_col not in d.columns or menu_col not in d.columns:
            if combined_col in d.columns:
                sp = d[combined_col].astype(str).str.split('_', n=1, expand=True)
                d[shop_col] = sp[0]
                d[menu_col] = sp[1] if sp.shape[1] > 1 else ''
            else:
                raise KeyError("영업장/메뉴 컬럼이 없어 병합 키를 만들 수 없습니다.")
        return d

    df1 = ensure_shop_menu_columns(df1)
    df  = ensure_shop_menu_columns(df)

    # 2) 가져올 컬럼 존재 여부 체크
    cols_available = [c for c in cols_to_add if c in df.columns]
    if not cols_available:
        raise KeyError(f"df에 가져올 컬럼이 없습니다: {cols_to_add}")

    # 3) df에서 키 중복 제거(최근/마지막 값 우선)
    key_cols = [shop_col, menu_col, date_col]
    df_pull = (df[key_cols + cols_available]
               .sort_values(key_cols)
               .drop_duplicates(subset=key_cols, keep='last'))

    # 4) 머지
    suffix = "_from_df"
    merged = df1.merge(df_pull, on=key_cols, how='left',
                       suffixes=('', suffix))

    # 5) 우선순위에 따라 값 결합
    for c in cols_available:
        src_col = c + suffix
        if src_col not in merged.columns:
            continue

        # 정수/불리언 성격 컬럼은 우선 float로 결합 후 캐스팅
        if prefer_df1:
            # df1 값이 우선: df1 값이 결측이면 df 값으로 채움
            merged[c] = merged[c].combine_first(merged[src_col])
        else:
            # df 값이 우선: df 값이 있으면 덮어씀
            merged[c] = merged[src_col].combine_first(merged[c])

        # 타입 정리(가능하면 0/1 정수로)
        if pd.api.types.is_bool_dtype(merged[c]):
            merged[c] = merged[c].astype('int8')
        else:
            # 값이 0/1 또는 결측이라면 0으로 채우고 정수화
            if set(pd.unique(merged[c].dropna())) <= {0,1}:
                merged[c] = merged[c].fillna(0).astype('int8')

        merged.drop(columns=[src_col], inplace=True)

    return merged

# ───────── 사용 예시 ─────────
df1 = add_off_and_zero_flags(
    df1, df,
    date_col='영업일자',
    shop_col='영업장명',
    menu_col='메뉴명',
    combined_col='영업장명_메뉴명',
    cols_to_add=('휴무일','custom_offday','is_zero_sales_period'),
    prefer_df1=True   # df1 값 보존, 빈 곳만 df로 보완
)

# 적용 확인 (간단 점검)
for c in ['휴무일','custom_offday','is_zero_sales_period']:
    if c in df1.columns:
        print(c, "value counts ->")
        print(df1[c].value_counts(dropna=False).head())


휴무일 value counts ->
휴무일
0    95617
1     7059
Name: count, dtype: int64
custom_offday value counts ->
custom_offday
0    96633
1     6043
Name: count, dtype: int64
is_zero_sales_period value counts ->
is_zero_sales_period
0    72818
1    29858
Name: count, dtype: int64


In [ ]:
# '영업장명'과 '메뉴명' 결합
df1['영업장명_메뉴명'] = df1['영업장명'].astype(str) + '_' + df1['메뉴명'].astype(str)
df1['영업일자'] = pd.to_datetime(df1['영업일자'])  # 문자열 → datetime 변환
df1['year'] = df1['영업일자'].dt.year
df1['month'] = df1['영업일자'].dt.month
df1['day'] = df1['영업일자'].dt.day
df1['weekday'] = pd.to_datetime(df1['영업일자']).dt.weekday
# 2. is_weekend 생성 (금~일 = 5, 6)
df1['is_weekend'] = df1['weekday'].astype(int).isin([5, 6]).astype(int)

In [ ]:
import pandas as pd
import holidays

# 1) 날짜 캘린더 생성
date_list = pd.date_range(start='2023-01-01', end='2024-06-15', freq='D')

# 2) 한국 공휴일 객체 (연도 지정해두면 빠름/안전)
years = list(range(date_list.year.min(), date_list.year.max() + 1))
kr_holidays = holidays.KR(years=years)

# 3) 캘린더 DF + 공휴일 플래그/전후일 플래그
holiday_df = pd.DataFrame({'ds': date_list})
holiday_df['holiday'] = holiday_df['ds'].apply(lambda x: 'holiday' if x in kr_holidays else 'non-holiday')
holiday_df['is_holiday'] = (holiday_df['holiday'] == 'holiday').astype('int8')

holiday_df = holiday_df.sort_values('ds')
holiday_df['holiday_prev1'] = holiday_df['is_holiday'].shift(1).fillna(0).astype('int8')
holiday_df['holiday_next1'] = holiday_df['is_holiday'].shift(-1).fillna(0).astype('int8')

# 4) 원본 df1에 병합
df1 = df1.merge(
    holiday_df[['ds', 'holiday', 'is_holiday', 'holiday_prev1', 'holiday_next1']],
    how='left', left_on='영업일자', right_on='ds'
)

# 5) 정리
df1.drop(columns=['ds'], inplace=True)
# (선택) 숫자형 일관성 유지
df1['holiday_prev1']  = pd.to_numeric(df1['holiday_prev1'], errors='coerce').fillna(0).astype('int8')
df1['holiday_next1']  = pd.to_numeric(df1['holiday_next1'], errors='coerce').fillna(0).astype('int8')
df1['is_holiday']     = pd.to_numeric(df1['is_holiday'],  errors='coerce').fillna(0).astype('int8')
# 문자열 'holiday'/'non-holiday' 유지가 필요 없으면 아래 줄로 삭제 가능
df1.drop(columns=['holiday'], inplace=True)


In [ ]:
df1.head()

,영업장명,메뉴명,영업일자,매출수량,휴무일,custom_offday,is_zero_sales_period,영업장명_메뉴명,year,month,day,weekday,is_weekend,is_holiday,holiday_prev1,holiday_next1
0,느티나무 셀프BBQ,1인 수저세트,2023-01-01,0.0,0,0,1,느티나무 셀프BBQ_1인 수저세트,2023,1,1,6,1,1,0,0
1,느티나무 셀프BBQ,1인 수저세트,2023-01-02,0.0,0,1,1,느티나무 셀프BBQ_1인 수저세트,2023,1,2,0,0,0,1,0
2,느티나무 셀프BBQ,1인 수저세트,2023-01-03,0.0,0,0,1,느티나무 셀프BBQ_1인 수저세트,2023,1,3,1,0,0,0,0
3,느티나무 셀프BBQ,1인 수저세트,2023-01-04,0.0,0,0,1,느티나무 셀프BBQ_1인 수저세트,2023,1,4,2,0,0,0,0
4,느티나무 셀프BBQ,1인 수저세트,2023-01-05,0.0,0,0,1,느티나무 셀프BBQ_1인 수저세트,2023,1,5,3,0,0,0,0


In [ ]:
# 각 날짜별 전체 매출 합산
total_sales_per_day = df1.groupby('영업일자')['매출수량'].sum().reset_index()

# 매출수량이 0인 영업일자 필터링
zero_sales_dates = total_sales_per_day[total_sales_per_day['매출수량'] == 0]['영업일자']

print(f"📌 모든 조합에서 매출이 0인 영업일자 수: {len(zero_sales_dates)}일")
print(zero_sales_dates.tolist()[:5])  # 일부 출력

📌 모든 조합에서 매출이 0인 영업일자 수: 17일
[Timestamp('2023-03-02 00:00:00'), Timestamp('2023-03-03 00:00:00'), Timestamp('2023-03-04 00:00:00'), Timestamp('2023-03-05 00:00:00'), Timestamp('2023-03-06 00:00:00')]


In [ ]:
#영업장별 매출 수량이 상위권인 메뉴 표시
# ➊ 업장·메뉴별 누적 매출 계산
menu_sales = (
    df1
    .groupby(['영업장명', '영업장명_메뉴명'])['매출수량']
    .sum()
    .reset_index()
    .rename(columns={'매출수량': 'total_menu_sales'})
)

main_menu_flags = []

# ➋ 업장별로 주요 메뉴 식별 (비율 0.4, 또는 최고 매출의 0.5배 미만 기준)
for store, grp in menu_sales.groupby('영업장명'):
    grp = grp.sort_values('total_menu_sales', ascending=False).reset_index(drop=True)
    sales = grp['total_menu_sales'].values
    max_sales = sales[0]

    # consecutive 비율 변화 계산 (sales[i] / sales[i-1])
    ratios = sales[1:] / (sales[:-1] + 1e-6)

    # ① 비율이 0.4 미만인 최초 위치
    idx_ratio = np.where(ratios < 0.4)[0]
    cutoff_ratio = idx_ratio[0] + 1 if len(idx_ratio) > 0 else len(sales)

    # ② 최고 매출의 0.5배 미만이 되는 최초 위치
    idx_half = np.where(sales < 0.5 * max_sales)[0]
    cutoff_half = idx_half[0] if len(idx_half) > 0 else len(sales)

    # 최종 컷오프는 두 기준 중 더 작은 인덱스
    cutoff = min(cutoff_ratio, cutoff_half)

    main_menus = grp.loc[:cutoff-1, '영업장명_메뉴명'].tolist()

    for menu in grp['영업장명_메뉴명']:
        main_menu_flags.append({
            '영업장명_메뉴명': menu,
            'is_main_menu': int(menu in main_menus)
        })

main_menu_df = pd.DataFrame(main_menu_flags)

# ➌ df1에 병합
df1= df1.merge(
    main_menu_df,
    on='영업장명_메뉴명',
    how='left'
)

# ➍ NaN 처리 (없으면 0으로)
df1['is_main_menu'] = df1['is_main_menu'].fillna(0).astype(int)

시계열 데이터 초반에 60일 이상 연속 0 데이터일 시 신규 출시라고 가정

In [ ]:
import pandas as pd

# 1) 메뉴별 첫 판매일
first_sale = (df1[df1['매출수량'] > 0]
              .groupby('영업장명_메뉴명')['영업일자']
              .min()
              .rename('first_sale_date'))

# 2) 메뉴별 첫 관측일
first_seen = (df1.groupby('영업장명_메뉴명')['영업일자']
                .min()
                .rename('first_seen_date'))

# 3) 매장별(전체 메뉴 합) 일별 매출
store_daily = (df1.groupby(['영업장명','영업일자'])['매출수량']
                 .sum()
                 .reset_index()
                 .rename(columns={'매출수량':'store_sales'}))

# 4) 메뉴–매장 매핑
menu_store = df1[['영업장명_메뉴명','영업장명']].drop_duplicates()

# 5) 메뉴별 지표 집계
info = (menu_store
        .merge(first_sale, left_on='영업장명_메뉴명', right_index=True, how='left')
        .merge(first_seen, left_on='영업장명_메뉴명', right_index=True, how='left'))

# 출시 전 매장 가동 여부(해당 매장에서 첫판매 이전에 다른 매출 > 0이 있었는지)
def had_store_sales_before(row):
    if pd.isna(row['first_sale_date']):
        # 아예 판매가 없었던 메뉴: 신규 판단 불가
        return False
    s = store_daily[(store_daily['영업장명']==row['영업장명']) &
                    (store_daily['영업일자'] < row['first_sale_date']) &
                    (store_daily['store_sales'] > 0)]
    return len(s) > 0

info['store_active_before'] = info.apply(had_store_sales_before, axis=1)

# 출시 전 공백 일수
info['days_idle_before'] = (info['first_sale_date'] - info['first_seen_date']).dt.days

# 출시 후 초기 60일 동안의 비0 판매일 수
k_window = 60
k_min_nonzero = 5

def count_nonzero_after(menu):
    g = df1[df1['영업장명_메뉴명']==menu].sort_values('영업일자')
    fs = info.loc[info['영업장명_메뉴명']==menu, 'first_sale_date'].values[0]
    if pd.isna(fs):
        return 0
    sub = g[(g['영업일자']>=fs) & (g['영업일자']<fs + pd.Timedelta(days=k_window))]
    return int((sub['매출수량'] > 0).sum())

nz_after = {m: count_nonzero_after(m) for m in info['영업장명_메뉴명']}
info['nonzero_days_60d'] = info['영업장명_메뉴명'].map(nz_after)

# 최종: 신규 출시 추정
info['is_probable_launch'] = (
    info['first_sale_date'].notna() &
    info['store_active_before'] &
    (info['days_idle_before'] >= 60) &
    (info['nonzero_days_60d'] >= k_min_nonzero)
).astype(int)

# 결과 확인
launch_candidates = info[info['is_probable_launch']==1] \
    .sort_values(['영업장명','first_sale_date'])

In [ ]:
# info: (영업장명_메뉴명, 영업장명, first_sale_date, first_seen_date, store_active_before, days_idle_before, nonzero_days_60d, is_probable_launch)

# ① df에 출시 메타 병합
df1 = df1.merge(
    info[['영업장명_메뉴명','first_sale_date','first_seen_date','is_probable_launch']],
    on='영업장명_메뉴명', how='left'
)

# ② 출시 전/후 구분 및 경과일
df1['days_since_launch'] = (df1['영업일자'] - df1['first_sale_date']).dt.days
df1['is_prelaunch']      = df1['days_since_launch'].lt(0).astype(int)      # 출시 전(True=1)
df1['is_postlaunch']     = df1['days_since_launch'].ge(0).astype(int)      # 출시 후(True=1)

# ③ 출시 직후 콜드스타트 구간(예: 60일) 표시
COLD_DAYS = 60
df1['is_coldstart_60d'] = ((df1['days_since_launch'] >= 0) &
                          (df1['days_since_launch'] < COLD_DAYS)).astype(int)

# ⑤ 출시 전 구조적 0에 대한 명확한 마스크(학습 제외용)
df1['structural_zero'] = ((df1['is_prelaunch'] == 1) & (df1['매출수량'] == 0)).astype(int)

# ⑥ 판매 시작이 한 번도 없던 메뉴(NaT) 처리용 플래그
df1['never_sold'] = df1['first_sale_date'].isna().astype(int)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102676 entries, 0 to 102675
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   영업장명                  102676 non-null  object        
 1   메뉴명                   102676 non-null  object        
 2   영업일자                  102676 non-null  datetime64[ns]
 3   매출수량                  102676 non-null  float64       
 4   휴무일                   102676 non-null  int8          
 5   custom_offday         102676 non-null  int8          
 6   is_zero_sales_period  102676 non-null  int64         
 7   영업장명_메뉴명              102676 non-null  object        
 8   year                  102676 non-null  int32         
 9   month                 102676 non-null  int32         
 10  day                   102676 non-null  int32         
 11  weekday               102676 non-null  int32         
 12  is_weekend            102676 non-null  int64         
 13 

In [ ]:
import pandas as pd
import numpy as np

def add_menu_month_features(
    df: pd.DataFrame,
    date_col="영업일자",
    shop_col="영업장명",
    menu_col="메뉴명",
    combined_col="영업장명_메뉴명",
    sales_col="매출수량",
    # 임계/파라미터
    top_k=3, bottom_k=3,          # MOY 상/하위 개수
    good_month_quantile=0.75,     # “좋은 달” 전월 롤링 분위
    rolling_q_window=12,          # 분위 롤링 윈도우(개월)
    sharp_drop_pct=-0.40,         # 전월 vs 전전월 급락 임계(%)
    min_abs_drop=20,              # 급락 절대량 임계
    min_prev=30,                  # 전전월 최소 규모
    share_threshold=0.05,         # 낮은 기여 임계 (전월 누적점유율)
    min_total_sales=1,            # 전월까지 누적합 최소
    min_obs_moy=2                 # MOY 평균 산출 최소 관측수(전월 시점까지)
):
    """
    ✅ 누수 방지 버전: '전월 기준(lag1)'으로만 월 지표를 만들어 일(행) 단위로 붙입니다.

    생성 컬럼(모두 lag1, 전월 기준):
      - lag1_is_high_season_moy, lag1_is_low_season_moy
      - lag1_is_good_month_quantile, lag1_is_sharp_drop_month
      - lag1_is_low_share_month, lag1_month_share
      - lag1_monthly_sales (보조 지표: 전월 월합)
    """
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col])

    # 키 확보
    if shop_col not in out.columns or menu_col not in out.columns:
        if combined_col in out.columns:
            pair = out[combined_col].astype(str).str.split('_', n=1, expand=True)
            out[shop_col] = pair[0]
            out[menu_col] = pair[1] if pair.shape[1] > 1 else ''
        else:
            raise KeyError("영업장/메뉴 키가 없어 피처를 생성할 수 없습니다.")

    # 월 period
    if ('월_기간' not in out.columns) or (not isinstance(out['월_기간'].dtype, pd.PeriodDtype)):
        out['월_기간'] = out[date_col].dt.to_period('M')

    key_pair = [shop_col, menu_col]
    key = key_pair + ['월_기간']

    # ── 1) 메뉴×월 월합 ─────────────────────────────────────
    monthly = (
        out.groupby(key, as_index=False)[sales_col]
           .sum()
           .rename(columns={sales_col: 'monthly_sales'})
           .sort_values(key)
    )
    monthly['month']   = monthly['월_기간'].dt.month
    monthly['m_prev']  = monthly.groupby(key_pair)['monthly_sales'].shift(1)
    monthly['m_prev2'] = monthly.groupby(key_pair)['monthly_sales'].shift(2)
    monthly['cum_prev'] = monthly.groupby(key_pair)['monthly_sales'].cumsum().shift(1)

    # ── 2) 전월 누적 점유율 & 낮은 기여달 ─────────────────────
    monthly['lag1_month_share'] = (monthly['m_prev'] / monthly['cum_prev'])
    monthly['lag1_month_share'] = monthly['lag1_month_share'] \
        .replace([np.inf, -np.inf], np.nan).fillna(0.0).astype('float32')

    monthly['lag1_is_low_share_month'] = (
        (monthly['cum_prev'] >= float(min_total_sales)) &
        (monthly['lag1_month_share'] <= float(share_threshold))
    ).astype('int8')

    # ── 3) 전월 “좋은 달”: 전월까지 롤링 분위와 비교 ───────────
    monthly['q_thr_prev'] = monthly.groupby(key_pair)['monthly_sales'] \
        .transform(lambda s: s.shift(1).rolling(rolling_q_window, min_periods=3)
                   .quantile(good_month_quantile))
    monthly['lag1_is_good_month_quantile'] = (
        (monthly['m_prev'] >= monthly['q_thr_prev'])
    ).fillna(0).astype('int8')

    # ── 4) 전월 급락(전월 vs 전전월) ─────────────────────────
    with np.errstate(divide='ignore', invalid='ignore'):
        pct = (monthly['m_prev'] - monthly['m_prev2']) / monthly['m_prev2']
    abs_change = (monthly['m_prev'] - monthly['m_prev2'])
    cond = monthly['m_prev2'].notna() & (monthly['m_prev2'] >= min_prev)
    monthly['lag1_is_sharp_drop_month'] = (
        cond & (pct <= sharp_drop_pct) & (abs_change.abs() >= min_abs_drop)
    ).astype('int8')

    # ── 5) 전월 MOY 시즌성(전월 시점까지의 MOY 누적평균으로 순위화) ───
    #   각 월(1..12)에 대해 누적합/누적건수 → 전월 시점 평균
    for mo in range(1, 13):
        mask = (monthly['month'] == mo).astype('int8')
        monthly[f'cum_sum_m{mo}'] = (monthly['monthly_sales'] * mask).cumsum().shift(1)
        monthly[f'cum_cnt_m{mo}'] = (mask).cumsum().shift(1)

    monthly['month_prev'] = monthly['month'].shift(1)

    def _rank_moy_row(row):
        means = []
        for mo in range(1, 13):
            s = row.get(f'cum_sum_m{mo}', np.nan)
            c = row.get(f'cum_cnt_m{mo}', np.nan)
            means.append(np.nan if (pd.isna(c) or c < min_obs_moy) else (s / c if c else np.nan))
        means = np.array(means, dtype='float64')
        valid = ~np.isnan(means)
        if pd.isna(row['month_prev']) or not valid.any():
            return 0, 0
        idx_prev = int(row['month_prev']) - 1
        if not valid[idx_prev]:
            return 0, 0
        k_high = min(top_k, valid.sum())
        k_low  = min(bottom_k, valid.sum())
        # 상위 k
        vals_high = np.where(valid, means, -np.inf)
        top_idx   = np.argsort(vals_high)[-k_high:] if k_high > 0 else np.array([], dtype=int)
        high_flag = int(idx_prev in top_idx)
        # 하위 k
        vals_low  = np.where(valid, means, np.inf)
        bot_idx   = np.argsort(vals_low)[:k_low] if k_low > 0 else np.array([], dtype=int)
        low_flag  = int(idx_prev in bot_idx)
        return high_flag, low_flag

    high_low = monthly.apply(lambda r: _rank_moy_row(r), axis=1, result_type='expand')
    monthly['lag1_is_high_season_moy'] = high_low.iloc[:, 0].astype('int8')
    monthly['lag1_is_low_season_moy']  = high_low.iloc[:, 1].astype('int8')

    # 보조: 전월 월합
    monthly['lag1_monthly_sales'] = monthly['m_prev'].fillna(0).astype('float32')

    # ── 6) 일(행) 단위로 브로드캐스트 ─────────────────────────
    attach = [
        'lag1_month_share',
        'lag1_is_low_share_month',
        'lag1_is_good_month_quantile',
        'lag1_is_sharp_drop_month',
        'lag1_is_high_season_moy',
        'lag1_is_low_season_moy',
        'lag1_monthly_sales'
    ]
    out = out.merge(monthly[key + attach], on=key, how='left')

    # 결측/타입 마무리
    for c in attach:
        if c.endswith('month_share') or c.endswith('monthly_sales'):
            out[c] = out[c].fillna(0.0).astype('float32')
        else:
            out[c] = out[c].fillna(0).astype('int8')

    return out

In [ ]:
df1 = add_menu_month_features(
    df1,
    date_col="영업일자",
    shop_col="영업장명",
    menu_col="메뉴명",
    sales_col="매출수량",
    top_k=3, bottom_k=3,
    good_month_quantile=0.75,
    rolling_q_window=12,
    sharp_drop_pct=-0.40,
    min_abs_drop=20,
    min_prev=30,
    share_threshold=0.05
)


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102676 entries, 0 to 102675
Data columns (total 34 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   영업장명                         102676 non-null  object        
 1   메뉴명                          102676 non-null  object        
 2   영업일자                         102676 non-null  datetime64[ns]
 3   매출수량                         102676 non-null  float64       
 4   휴무일                          102676 non-null  int8          
 5   custom_offday                102676 non-null  int8          
 6   is_zero_sales_period         102676 non-null  int64         
 7   영업장명_메뉴명                     102676 non-null  object        
 8   year                         102676 non-null  int32         
 9   month                        102676 non-null  int32         
 10  day                          102676 non-null  int32         
 11  weekday                   

In [ ]:
import pandas as pd
import numpy as np

def add_low_share_month_feature_leak_safe(
    df: pd.DataFrame,
    *,
    date_col: str = '영업일자',
    shop_col: str = '영업장명',
    menu_col: str = '메뉴명',
    combined_col: str = '영업장명_메뉴명',
    sales_col: str = '매출수량',
    period_col: str = '월_기간',
    share_threshold: float = 0.05,          # 임계: ≤ 5%
    min_total_sales: float | int = 1,       # 누적합 최소
    overwrite: bool = True                  # 동일 컬럼 존재 시 덮어쓸지
) -> pd.DataFrame:
    """
    학습 안전(누수 방지) 버전:
      - lag1_month_share = (전월 매출 / 전월까지의 누적합)
      - lag1_is_low_share_month = (lag1_month_share ≤ 임계) & (누적합 ≥ min_total_sales)
    일(행) 단위 DF에 월 키로 브로드캐스트합니다.
    """
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col])

    # (영업장, 메뉴) 확보
    if (shop_col not in out.columns) or (menu_col not in out.columns):
        if combined_col in out.columns:
            sp = out[combined_col].astype(str).str.split('_', n=1, expand=True)
            out[shop_col] = sp[0]
            out[menu_col] = sp[1] if sp.shape[1] > 1 else ''
        else:
            raise KeyError("영업장/메뉴 키가 없어 피처를 생성할 수 없습니다.")

    # 월 period
    if (period_col not in out.columns) or (not pd.api.types.is_period_dtype(out[period_col])):
        out[period_col] = out[date_col].dt.to_period('M')

    key_pair = [shop_col, menu_col]
    key = key_pair + [period_col]

    # (영업장,메뉴,월) 총매출
    monthly = (
        out.groupby(key, as_index=False)[sales_col]
           .sum()
           .rename(columns={sales_col: 'monthly_sales'})
           .sort_values(key)
    )

    # 전월/전월까지 누적합
    monthly['m_prev']   = monthly.groupby(key_pair)['monthly_sales'].shift(1)
    monthly['cum_prev'] = monthly.groupby(key_pair)['monthly_sales'].cumsum().shift(1)

    # 전월 점유율 & 낮은 기여 플래그
    with np.errstate(divide='ignore', invalid='ignore'):
        lag_share = monthly['m_prev'] / monthly['cum_prev']
    monthly['lag1_month_share'] = (
        pd.Series(lag_share).replace([np.inf, -np.inf], np.nan).fillna(0.0).astype('float32')
    )
    monthly['lag1_is_low_share_month'] = (
        (monthly['cum_prev'] >= float(min_total_sales)) &
        (monthly['lag1_month_share'] <= float(share_threshold))
    ).astype('int8')

    # 일(행) 단위로 브로드캐스트
    attach = ['lag1_month_share', 'lag1_is_low_share_month']
    merged = out.merge(monthly[key + attach], on=key, how='left', suffixes=('', '_calc'))

    # 덮어쓰기/결측 처리
    for col, dtype, fill in [('lag1_month_share', 'float32', 0.0),
                             ('lag1_is_low_share_month', 'int8', 0)]:
        src = col + '_calc'
        if src in merged.columns:
            if overwrite or col not in merged.columns:
                merged[col] = merged[src]
            else:
                merged[col] = merged[col].combine_first(merged[src])
            merged.drop(columns=[src], inplace=True)
        merged[col] = merged[col].fillna(fill).astype(dtype)

    return merged


In [ ]:
df1 = add_low_share_month_feature_leak_safe(
    df1,
    share_threshold=0.05,
    min_total_sales=1,
    overwrite=True
)


/tmp/ipython-input-2447785421.py:36: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if (period_col not in out.columns) or (not pd.api.types.is_period_dtype(out[period_col])):


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102676 entries, 0 to 102675
Data columns (total 34 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   영업장명                         102676 non-null  object        
 1   메뉴명                          102676 non-null  object        
 2   영업일자                         102676 non-null  datetime64[ns]
 3   매출수량                         102676 non-null  float64       
 4   휴무일                          102676 non-null  int8          
 5   custom_offday                102676 non-null  int8          
 6   is_zero_sales_period         102676 non-null  int64         
 7   영업장명_메뉴명                     102676 non-null  object        
 8   year                         102676 non-null  int32         
 9   month                        102676 non-null  int32         
 10  day                          102676 non-null  int32         
 11  weekday                   

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

LOOKBACK = 28
HORIZON  = 7
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-3
EMBED_DIM = 8        # 메뉴 임베딩 차원
HIDDEN = 64
NUM_LAYERS = 1
DROPOUT = 0.1

# ==== 1) 유틸: 손실/지표 ====
class SMAPE_MAE_Loss(nn.Module):
    def __init__(self, alpha=0.8):
        super().__init__()
        self.alpha = alpha
    def forward(self, pred, target):
        smape = torch.mean(2 * torch.abs(pred - target) / (torch.abs(pred) + torch.abs(target) + 1e-8))
        mae = torch.mean(torch.abs(pred - target))
        return self.alpha * smape + (1 - self.alpha) * mae

@torch.no_grad()
def smape_mae(pred, target):
    smape = torch.mean(2 * torch.abs(pred - target) / (torch.abs(pred) + torch.abs(target) + 1e-8)).item()
    mae = torch.mean(torch.abs(pred - target)).item()
    return smape, mae

In [ ]:
# ==== 2) 데이터셋 생성 ====
class SeqDataset(Dataset):
    def __init__(self, X, y, menu_ids):
        self.X = X     # (N, LOOKBACK, F)
        self.y = y     # (N, HORIZON)
        self.menu_ids = menu_ids  # (N,)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.menu_ids[idx]

def build_store_datasets(store_df: pd.DataFrame, exog_cols):
    """
    매장 내 각 메뉴의 시계열로부터 시퀀스를 만들어
    (train_dataset, val_dataset, scalers, menu_id_map, feature_dim) 반환
    """
    X_tr_list, y_tr_list, mid_tr_list = [], [], []
    X_va_list, y_va_list, mid_va_list = [], [], []

    # 메뉴 ID 부여
    menus = store_df['메뉴명'].dropna().unique().tolist()
    menu2id = {m:i for i,m in enumerate(sorted(menus))}
    n_menus = len(menu2id)

    # 메뉴별 스케일러(매출수량만 스케일링)
    scalers = {}

    for menu, g in store_df.groupby('메뉴명'):
        g = g.sort_values('영업일자').copy()
        # 필요한 컬럼 보정
        g['매출수량'] = g['매출수량'].fillna(0)

        # 검증 기준 인덱스(마지막 7일)
        # 시퀀스 슬라이딩 시, 타깃 구간의 끝이 split_idx를 넘으면 검증으로 보냄
        split_date = g['영업일자'].max() - pd.Timedelta(days=HORIZON-1)
        split_idx = g.index[g['영업일자'] >= split_date][0]  # 마지막 7일의 시작 위치 인덱스값

        train_mask = g.index < split_idx           # 마지막 7일 시작 index 이전만
        scaler = MinMaxScaler().fit(g.loc[train_mask, ['매출수량']])
        g['scaled_sales'] = scaler.transform(g[['매출수량']])
        scalers[menu] = scaler

        # 시계열 배열 구성
        feats = ['scaled_sales'] + exog_cols
        arr = g[feats].fillna(0).values.astype('float32')
        mid = np.full((len(g),), menu2id[menu], dtype='int64')

        # 시퀀스 만들기
        # i ... i+LOOKBACK-1 -> 입력, i+LOOKBACK ... i+LOOKBACK+HORIZON-1 -> 타깃
        for i in range(len(g) - LOOKBACK - HORIZON + 1):
            X_seq = arr[i:i+LOOKBACK, :]
            y_seq = g['scaled_sales'].values[i+LOOKBACK:i+LOOKBACK+HORIZON]
            m_id  = mid[i]

            # 타깃 구간의 마지막 시점
            target_end_pos = g.index[i+LOOKBACK+HORIZON-1]
            if target_end_pos < split_idx:
                X_tr_list.append(X_seq)
                y_tr_list.append(y_seq)
                mid_tr_list.append(m_id)
            else:
                X_va_list.append(X_seq)
                y_va_list.append(y_seq)
                mid_va_list.append(m_id)

    def to_tensor(xl):
        return torch.tensor(np.stack(xl)).float() if len(xl)>0 else torch.empty(0)

    X_tr = to_tensor(X_tr_list)
    y_tr = to_tensor(y_tr_list)
    X_va = to_tensor(X_va_list)
    y_va = to_tensor(y_va_list)
    mid_tr = torch.tensor(np.array(mid_tr_list), dtype=torch.long) if len(mid_tr_list)>0 else torch.empty(0, dtype=torch.long)
    mid_va = torch.tensor(np.array(mid_va_list), dtype=torch.long) if len(mid_va_list)>0 else torch.empty(0, dtype=torch.long)

    train_ds = SeqDataset(X_tr, y_tr, mid_tr)
    valid_ds = SeqDataset(X_va, y_va, mid_va)

    feature_dim = X_tr.shape[-1] if len(X_tr_list)>0 else (X_va.shape[-1] if len(X_va_list)>0 else len(['scaled_sales']+exog_cols))
    return train_ds, valid_ds, scalers, menu2id, feature_dim

In [ ]:
# ==== 3) 모델 ====
class StoreLSTM(nn.Module):
    def __init__(self, n_menus, feature_dim, embed_dim=8, hidden=128, num_layers=1, dropout=0.1, horizon=7):
        super().__init__()
        self.embed = nn.Embedding(num_embeddings=max(n_menus,1), embedding_dim=embed_dim)
        self.lstm  = nn.LSTM(input_size=feature_dim + embed_dim,
                             hidden_size=hidden,
                             num_layers=num_layers,
                             batch_first=True,
                             dropout=dropout if num_layers>1 else 0.0)
        self.head  = nn.Sequential(
            nn.Linear(hidden, hidden//2),
            nn.ReLU(),
            nn.Linear(hidden//2, horizon)
        )
    def forward(self, x, menu_id):
        # x: (B, L, F), menu_id: (B,)
        emb = self.embed(menu_id)                     # (B, E)
        emb = emb.unsqueeze(1).expand(-1, x.size(1), -1)  # (B, L, E)
        x_in = torch.cat([x, emb], dim=2)             # (B, L, F+E)
        out, (h, c) = self.lstm(x_in)
        last = out[:, -1, :]                          # (B, H)
        yhat = self.head(last)                        # (B, HORIZON)
        return yhat


In [ ]:
# ==== 4) 학습 루프 ====
def train_one_store(store_name, store_df, exog_cols):
    print(f"\n[Train] 매장: {store_name}")
    train_ds, valid_ds, scalers, menu2id, feature_dim = build_store_datasets(store_df, exog_cols)
    n_menus = len(menu2id)

    if len(train_ds)==0 or len(valid_ds)==0:
        print("  ↳ 시퀀스가 부족하여 스킵 (train/valid 빈 데이터)")
        return None

    model = StoreLSTM(n_menus=n_menus, feature_dim=feature_dim,
                      embed_dim=EMBED_DIM, hidden=HIDDEN, num_layers=NUM_LAYERS,
                      dropout=DROPOUT, horizon=HORIZON).to(DEVICE)
    crit = SMAPE_MAE_Loss(alpha=0.8)
    opt  = torch.optim.Adam(model.parameters(), lr=LR)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

    best_val = np.inf
    best_state = None

    for epoch in range(1, EPOCHS+1):
        model.train()
        tr_losses = []
        for Xb, yb, mb in train_loader:
            Xb = Xb.to(DEVICE)
            yb = yb.to(DEVICE)
            mb = mb.to(DEVICE)
            opt.zero_grad()
            pred = model(Xb, mb)
            loss = crit(pred, yb)
            loss.backward()
            opt.step()
            tr_losses.append(loss.item())

        # validation
        model.eval()
        va_losses, all_smape, all_mae = [], [], []
        with torch.no_grad():
            for Xb, yb, mb in valid_loader:
                Xb = Xb.to(DEVICE)
                yb = yb.to(DEVICE)
                mb = mb.to(DEVICE)
                pred = model(Xb, mb)
                loss = crit(pred, yb)
                va_losses.append(loss.item())
                s, m = smape_mae(pred, yb)
                all_smape.append(s); all_mae.append(m)

        tr_loss = np.mean(tr_losses) if tr_losses else np.nan
        va_loss = np.mean(va_losses) if va_losses else np.nan
        va_smape = np.mean(all_smape) if all_smape else np.nan
        va_mae   = np.mean(all_mae) if all_mae else np.nan
        print(f"  Epoch {epoch:02d} | train {tr_loss:.4f} | valid {va_loss:.4f} | SMAPE {va_smape:.4f} | MAE {va_mae:.4f}")

        if va_loss < best_val:
            best_val = va_loss
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}

    if best_state is not None:
        model.load_state_dict(best_state)

    return {
        "model": model,
        "scalers": scalers,
        "menu2id": menu2id,
        "exog_cols": exog_cols,
        "feature_dim": feature_dim,
        "store": store_name
    }


In [ ]:
import pandas as pd

# ==== 5) 전체 매장 학습 (누수-안전 lag 피처 사용) ====
def train_lstm_per_store(df1: pd.DataFrame, drop_leaky: bool = True):
    # 0) 필수 컬럼 체크
    needed = {'영업일자','영업장명','영업장명_메뉴명','메뉴명','매출수량'}
    missing = needed - set(df1.columns)
    if missing:
        raise ValueError(f"다음 컬럼이 필요합니다: {missing}")

    df = df1.copy()
    df['영업일자'] = pd.to_datetime(df['영업일자'])

    # 1) 기본 파생
    if 'weekday' not in df.columns:
        df['weekday'] = df['영업일자'].dt.weekday
    if 'is_weekend' not in df.columns:
        df['is_weekend'] = df['weekday'].isin([5,6]).astype(int)
    if 'month' not in df.columns:
        df['month'] = df['영업일자'].dt.month
    if '휴무일' not in df.columns:
        df['휴무일'] = 0
    if 'custom_offday' not in df.columns:
        df['custom_offday'] = 0

    # 2) 원-핫: 항상 고정 차원(weekday=0..6, month=1..12)
    df['weekday'] = pd.Categorical(df['weekday'].astype(int), categories=list(range(7)), ordered=True)
    df['month']   = pd.Categorical(df['month'].astype(int),   categories=list(range(1,13)), ordered=True)

    wd = pd.get_dummies(df['weekday'], prefix='wd', dtype='int8')
    m  = pd.get_dummies(df['month'],   prefix='m',  dtype='int8')
    # 누락된 더미가 있으면 채워 넣기(이론상 위 Categorical로 이미 모두 생성됨)
    for c in [f'wd_{i}' for i in range(7)]:
        if c not in wd.columns: wd[c] = 0
    for c in [f'm_{i}' for i in range(1,13)]:
        if c not in m.columns: m[c] = 0

    wd = wd[[f'wd_{i}' for i in range(7)]]
    m  = m[[f'm_{i}'  for i in range(1,13)]]

    df = pd.concat([df, wd, m], axis=1)
    wd_cols = list(wd.columns)
    m_cols  = list(m.columns)

    # 3) 사용할 외생 변수(존재하는 것만 채택)
    #    - 누수-안전 lag1 계열만 명시
    lag_safe_candidates = [
        # 공휴일
        'is_holiday', 'holiday_prev1', 'holiday_next1',
        # 런칭/수명주기 (날짜로 결정되는 지표라 누수 아님)
        'is_probable_launch',
        # 운영 플래그
        'custom_offday', '휴무일',
        # 메뉴 중요도
        'is_main_menu',
        # 월 지표(모두 lag1만 사용)
        'lag1_is_high_season_moy',
        'lag1_is_good_month_quantile',
        'lag1_is_low_share_month', 'lag1_month_share',
        # 선택: 월합 변화의 lag1 (있을 때만)
        'lag1_monthly_sales', 'lag1_monthly_abs_change', 'lag1_monthly_pct_change',
    ]

    # 실제 존재하는 컬럼만 채택 + 원-핫 더미 추가
    exog_cols = [c for c in lag_safe_candidates if c in df.columns] + wd_cols + m_cols

    # 4) 매장별 학습
    trained = {}
    for store, g in df.groupby('영업장명', sort=False):
        g = g.sort_values('영업일자')
        # 여기서 결측이 있다면 0으로 채움(대부분 lag1_*는 생성 시 0으로 채워져 있음)
        X_cols = [c for c in exog_cols if c in g.columns]
        g[X_cols] = g[X_cols].fillna(0)

        # train_one_store(store, g, exog_cols) 는 기존 함수 시그니처를 그대로 사용
        result = train_one_store(store, g, X_cols)
        if result is not None:
            trained[store] = result
    return trained


In [ ]:
if 'holiday' in df1.columns:
    df1['holiday'] = (
        df1['holiday']
        .replace({'holiday': 1, 'non-holiday': 0})
        .fillna(0)
        .astype(int)
    )
trained_models = train_lstm_per_store(df1)

In [ ]:
import pandas as pd
import numpy as np

# ---------------------------------------------
# 공통 유틸: 키/날짜 보정
# ---------------------------------------------
def _ensure_keys_and_date(df, date_col='영업일자', shop_col='영업장명', menu_col='메뉴명', combined_col='영업장명_메뉴명'):
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col])
    if (shop_col not in out.columns) or (menu_col not in out.columns):
        if combined_col in out.columns:
            sp = out[combined_col].astype(str).str.split('_', n=1, expand=True)
            out[shop_col] = sp[0]
            out[menu_col] = sp[1] if sp.shape[1] > 1 else ''
        else:
            raise KeyError("훈련 DF에 ('영업장명','메뉴명') 또는 '영업장명_메뉴명'이 필요합니다.")
    return out

# ---------------------------------------------
# 1) first_sale_table: 출시일(첫 판매일) 표
#    -> preprocess_test_df_for_inference(first_sale_table=...)에 사용
# ---------------------------------------------
def build_first_sale_table(train_df: pd.DataFrame,
                           date_col='영업일자', shop_col='영업장명', menu_col='메뉴명',
                           sales_col='매출수량') -> pd.DataFrame:
    d = _ensure_keys_and_date(train_df, date_col, shop_col, menu_col)
    d = d.sort_values([shop_col, menu_col, date_col])
    # 첫 판매일(매출수량 > 0)
    first_sale = (d[d[sales_col] > 0]
                    .groupby([shop_col, menu_col], as_index=False)[date_col]
                    .min()
                    .rename(columns={date_col: 'first_sale_date'}))
    # 영업장명_메뉴명도 같이 제공(머지 유연성)
    first_sale['영업장명_메뉴명'] = first_sale[shop_col].astype(str) + '_' + first_sale[menu_col].astype(str)
    return first_sale

# ---------------------------------------------
# 2) main_menu_df: 매장별 주요 메뉴 집합
#    (기본 규칙: 누적 매출 내림차순에서
#     - 연속비 0.4 미만 최초 지점까지,
#     - 또는 최고 매출의 0.5배 미만 최초 지점까지
#     중 더 엄격한 컷 사용)
#    -> preprocess_test_df_for_inference(main_menu_df=...)에 사용
# ---------------------------------------------
def build_main_menu_df(train_df: pd.DataFrame,
                       date_col='영업일자', shop_col='영업장명', menu_col='메뉴명',
                       sales_col='매출수량',
                       ratio_gap=0.4, half_of_max=0.5) -> pd.DataFrame:
    d = _ensure_keys_and_date(train_df, date_col, shop_col, menu_col)
    menu_sales = (d.groupby([shop_col, menu_col], as_index=False)[sales_col]
                    .sum()
                    .rename(columns={sales_col: 'total_menu_sales'}))
    rows = []
    for store, g in menu_sales.groupby(shop_col, sort=False):
        g = g.sort_values('total_menu_sales', ascending=False).reset_index(drop=True)
        sales = g['total_menu_sales'].values
        if len(sales) == 0:
            continue
        max_sales = sales[0]
        # 연속비(다음/이전)
        ratios = sales[1:] / (sales[:-1] + 1e-6)
        idx_ratio = np.where(ratios < ratio_gap)[0]
        cutoff_ratio = idx_ratio[0] + 1 if len(idx_ratio) > 0 else len(sales)
        # 최고 대비 0.5배 미만
        idx_half = np.where(sales < half_of_max * max_sales)[0]
        cutoff_half = idx_half[0] if len(idx_half) > 0 else len(sales)
        cutoff = min(cutoff_ratio, cutoff_half)
        main_menus = set(g.loc[:cutoff-1, menu_col].tolist())
        for _, r in g.iterrows():
            rows.append({
                '영업장명': store,
                '메뉴명': r[menu_col],
                '영업장명_메뉴명': f"{store}_{r[menu_col]}",
                'is_main_menu': int(r[menu_col] in main_menus)
            })
    out = pd.DataFrame(rows)
    if out.empty:
        return pd.DataFrame(columns=['영업장명','메뉴명','영업장명_메뉴명','is_main_menu'])
    out['is_main_menu'] = out['is_main_menu'].astype('int8')
    return out

# ---------------------------------------------
# 3) off_union_mday_df: 매장별 '연속 0매출(>=min_run일)' 구간의 (월,일) 합집합
#    -> preprocess_test_df_for_inference(off_union_mday_df=...)에 사용
# ---------------------------------------------
def build_off_union_mday_df(train_df: pd.DataFrame,
                            date_col='영업일자', store_col='영업장명',
                            sales_col='매출수량', min_run=4) -> pd.DataFrame:
    d = _ensure_keys_and_date(train_df, date_col, store_col, '메뉴명')
    # 매장×일자 총매출
    daily = (d.groupby([store_col, date_col], as_index=False)[sales_col]
               .sum()
               .rename(columns={sales_col: 'store_sales'}))
    rows = []
    for store, g in daily.groupby(store_col, sort=False):
        g = g.sort_values(date_col)
        full_idx = pd.date_range(g[date_col].min(), g[date_col].max(), freq='D')
        s = (g.set_index(date_col)['store_sales']
               .reindex(full_idx)
               .fillna(0.0))
        z = (s == 0).astype(int)
        run_id = (z.diff().fillna(z.iloc[0]) != 0).cumsum()
        tmp = (pd.DataFrame({'z': z, 'run_id': run_id})
                 .groupby('run_id', group_keys=False)
                 .apply(lambda d: pd.Series({
                     'flag': d['z'].iloc[0] == 1,
                     'start': d.index.min(),
                     'end': d.index.max(),
                     'len': len(d)
                 })))
        zero_runs = tmp[(tmp['flag']) & (tmp['len'] >= min_run)]
        if zero_runs.empty:
            continue
        for _, r in zero_runs.iterrows():
            days = pd.date_range(r['start'], r['end'], freq='D')
            rows.append(pd.DataFrame({
                store_col: store,
                'month': days.month,
                'day': days.day,
                'is_union_offday': 1
            }))
    if rows:
        out = (pd.concat(rows, ignore_index=True)
                 .drop_duplicates([store_col, 'month', 'day']))
    else:
        out = pd.DataFrame(columns=[store_col, 'month', 'day', 'is_union_offday'])
    out['is_union_offday'] = out.get('is_union_offday', 0).astype('int8')
    return out

# ---------------------------------------------
# 4) monthly_history: (영업장명, 메뉴명, 월_기간, monthly_sales)
#    -> predict 전처리에서 lag1 월 피처 계산에 사용
# ---------------------------------------------
def build_monthly_history(train_df: pd.DataFrame,
                          date_col='영업일자', shop_col='영업장명',
                          menu_col='메뉴명', sales_col='매출수량') -> pd.DataFrame:
    d = _ensure_keys_and_date(train_df, date_col, shop_col, menu_col)
    d['월_기간'] = d[date_col].dt.to_period('M')
    mh = (d.groupby([shop_col, menu_col, '월_기간'], as_index=False)[sales_col]
            .sum()
            .rename(columns={sales_col: 'monthly_sales'}))
    return mh


In [ ]:
# 훈련 DF 지정 (예: df1 또는 train_df)
train_df = df1  # 훈련 구간만 포함해야 안전합니다.

first_sale_table   = build_first_sale_table(train_df)
main_menu_df       = build_main_menu_df(train_df, ratio_gap=0.4, half_of_max=0.5)
off_union_mday_df  = build_off_union_mday_df(train_df, min_run=4)
monthly_history    = build_monthly_history(train_df)

# (원하시면 간단 점검)
print(first_sale_table.head(3))
print(main_menu_df['is_main_menu'].value_counts())
print(off_union_mday_df.head(3))
print(monthly_history.head(3))

/tmp/ipython-input-248179391.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({
/tmp/ipython-input-248179391.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({
/tmp/ipython-input-248179391.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the group

         영업장명          메뉴명 first_sale_date                영업장명_메뉴명
0  느티나무 셀프BBQ      1인 수저세트      2023-01-17      느티나무 셀프BBQ_1인 수저세트
1  느티나무 셀프BBQ    BBQ55(단체)      2023-01-03    느티나무 셀프BBQ_BBQ55(단체)
2  느티나무 셀프BBQ  대여료 30,000원      2023-01-01  느티나무 셀프BBQ_대여료 30,000원
is_main_menu
0    168
1     25
Name: count, dtype: int64
         영업장명  month  day  is_union_offday
0  느티나무 셀프BBQ      3    1                1
1  느티나무 셀프BBQ      3    2                1
2  느티나무 셀프BBQ      3    3                1
         영업장명      메뉴명     월_기간  monthly_sales
0  느티나무 셀프BBQ  1인 수저세트  2023-01           82.0
1  느티나무 셀프BBQ  1인 수저세트  2023-02          162.5
2  느티나무 셀프BBQ  1인 수저세트  2023-03           39.5


/tmp/ipython-input-248179391.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({
/tmp/ipython-input-248179391.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({


In [ ]:
# ==== 6) 예측 함수(각 매장의 모든 메뉴에 대해 7일 예측) ====
@torch.no_grad()
def predict_next_7days(df1: pd.DataFrame, trained_models: dict):
    df = df1.copy()
    df['영업일자'] = pd.to_datetime(df['영업일자'])
    preds = []

    for store, pack in trained_models.items():
        model = pack["model"]
        model.eval()
        scalers = pack["scalers"]
        menu2id = pack["menu2id"]
        exog_cols = pack["exog_cols"]

        sdf = df[df['영업장명']==store].copy()
        last_date = sdf['영업일자'].max()

        for menu, g in sdf.groupby('메뉴명'):
            if menu not in scalers or menu not in menu2id:
                continue
            g = g.sort_values('영업일자').copy()
            if len(g) < LOOKBACK:
                continue

            scaler = scalers[menu]
            g['scaled_sales'] = scaler.transform(g[['매출수량']])
            feats = ['scaled_sales'] + exog_cols
            arr = g[feats].fillna(0).values.astype('float32')

            X_seq = torch.tensor(arr[-LOOKBACK:], dtype=torch.float32).unsqueeze(0).to(DEVICE)
            m_id  = torch.tensor([menu2id[menu]], dtype=torch.long).to(DEVICE)
            yhat_scaled = model(X_seq, m_id).cpu().numpy().ravel()
            # 역스케일
            yhat = scaler.inverse_transform(yhat_scaled.reshape(-1,1)).ravel()

            # 예측 날짜 생성
            future_days = pd.date_range(last_date + pd.Timedelta(days=1), periods=HORIZON, freq='D')
            for d, val in zip(future_days, yhat):
                preds.append([store, menu, d, float(max(val, 0.0))])

    pred_df = pd.DataFrame(preds, columns=['영업장명','메뉴명','예측일자','예측수량'])
    return pred_df

In [ ]:
import pandas as pd
import numpy as np

def preprocess_test_df_for_inference(
    test_df: pd.DataFrame,
    exog_cols,
    *,
    # ==== 월 lag1 피처 계산용(훈련 데이터로만 만든 것 권장) ====
    monthly_history: pd.DataFrame | None = None,   # ['영업장명','메뉴명','월_기간','monthly_sales']

    # ==== 출시/메뉴/휴무 매핑(훈련에서 미리 만든 표 전달 권장) ====
    first_sale_table: pd.DataFrame | None = None,  # ['영업장명','메뉴명','first_sale_date']
    main_menu_df: pd.DataFrame | None = None,      # ['영업장명_메뉴명','is_main_menu'] 또는 ['영업장명','메뉴명','is_main_menu']
    off_union_mday_df: pd.DataFrame | None = None, # ['영업장명','month','day','is_union_offday']  (아래 helper 참조)

    # ==== lag 판정 파라미터(훈련과 동일 유지) ====
    share_threshold: float = 0.05,
    min_total_sales: float | int = 1,
    sharp_drop_pct: float = -0.40,
    min_abs_drop: float = 20,
    min_prev: float = 30,
    good_month_quantile: float = 0.75,
    rolling_q_window: int = 12,
    top_k: int = 3,
    bottom_k: int = 3,
    min_obs_moy: int = 2,
):
    """
    테스트 DF 전처리:
      - 키/날짜 파생 + 공휴일/전후일
      - wd_0..6, m_1..12 고정 원핫
      - 훈련 history 기반 lag1 월 피처
      - ✨ 출시 이후 플래그(is_probable_launch=1) 적용(= post-launch)
      - ✨ 매장별 '연속 0매출(>=4일)' 구간의 월-일 합집합으로 휴무일=1
      - ✨ 휴무일==0에서만 custom_offday 규칙 적용
      - ✨ 훈련에서 산정한 main_menu 집합 그대로 적용
      - exog_cols 중 누락은 0으로 보완
    """
    df = test_df.copy()

    # ── 0) 키/날짜 파생 ───────────────────────────────────────────
    if ('영업장명' not in df.columns) or ('메뉴명' not in df.columns):
        if '영업장명_메뉴명' not in df.columns:
            raise KeyError("테스트 DF에 '영업장명_메뉴명' 또는 ('영업장명','메뉴명')가 필요합니다.")
        sp = df['영업장명_메뉴명'].astype(str).str.split('_', n=1, expand=True)
        df['영업장명'] = sp[0]; df['메뉴명'] = sp[1] if sp.shape[1] > 1 else ''

    df['영업일자'] = pd.to_datetime(df['영업일자'])
    df['weekday']  = df['영업일자'].dt.weekday
    df['month']    = df['영업일자'].dt.month
    df['day']      = df['영업일자'].dt.day
    df['월_기간']    = df['영업일자'].dt.to_period('M')

    # ── 1) 공휴일/전후일 ──────────────────────────────────────────
    try:
        import holidays as pyholidays
        years = list(range(df['영업일자'].dt.year.min(), df['영업일자'].dt.year.max() + 1))
        kr = pyholidays.KR(years=years)
        cal = pd.DataFrame({'영업일자': pd.date_range(df['영업일자'].min(), df['영업일자'].max(), freq='D')})
        cal['is_holiday'] = cal['영업일자'].dt.date.map(lambda d: 1 if d in kr else 0).astype('int8')
    except Exception:
        cal = pd.DataFrame({'영업일자': pd.date_range(df['영업일자'].min(), df['영업일자'].max(), freq='D')})
        fixed_days = {(1,1),(3,1),(5,5),(6,6),(8,15),(10,3),(10,9),(12,25)}
        cal['is_holiday'] = cal['영업일자'].map(lambda d: 1 if (d.month, d.day) in fixed_days else 0).astype('int8')

    cal = cal.sort_values('영업일자')
    cal['holiday_prev1'] = cal['is_holiday'].shift(1).fillna(0).astype('int8')
    cal['holiday_next1'] = cal['is_holiday'].shift(-1).fillna(0).astype('int8')
    df = df.merge(cal, on='영업일자', how='left')

    # ── 2) 요일/월 원핫(고정 차원) ────────────────────────────────
    df['weekday'] = pd.Categorical(df['weekday'].astype(int), categories=list(range(7)), ordered=True)
    df['month']   = pd.Categorical(df['month'].astype(int),   categories=list(range(1,13)), ordered=True)
    wd = pd.get_dummies(df['weekday'], prefix='wd', dtype='int8')[[f'wd_{i}' for i in range(7)]]
    m  = pd.get_dummies(df['month'],   prefix='m',  dtype='int8')[[f'm_{i}' for i in range(1,13)]]
    df = pd.concat([df, wd, m], axis=1)

    # ── 3) lag1 월 피처(훈련 history로 계산) ───────────────────────
    lag_cols = [
        'lag1_month_share','lag1_is_low_share_month',
        'lag1_monthly_sales','lag1_monthly_abs_change','lag1_monthly_pct_change',
        'lag1_is_good_month_quantile',
        'lag1_is_high_season_moy',
    ]
    for c in lag_cols:
        if c not in df.columns:
            df[c] = 0  # 기본값

    if monthly_history is not None and len(monthly_history):
        mh = monthly_history.copy()
        if '월_기간' not in mh.columns:
            raise KeyError("monthly_history에는 '월_기간'(period[M]) 컬럼이 필요합니다.")
        if not isinstance(mh['월_기간'].dtype, pd.PeriodDtype):
            mh['월_기간'] = pd.PeriodIndex(mh['월_기간'], freq='M')
        for c in ['영업장명','메뉴명','월_기간','monthly_sales']:
            if c not in mh.columns:
                raise KeyError(f"monthly_history에 '{c}' 컬럼이 필요합니다.")

        feats_rows = []
        test_months = df[['영업장명','메뉴명','월_기간']].drop_duplicates()

        for (store, menu), g_hist in mh.groupby(['영업장명','메뉴명'], sort=False):
            s = g_hist.sort_values('월_기간').set_index('월_기간')['monthly_sales'].astype(float)
            if s.empty:
                continue
            cum = s.cumsum()
            s_shift = s.shift(1)

            # MOY 누적 평균 준비
            month_idx = pd.Index([p.month for p in s.index], name='m')
            cum_sum = {mo: (s.where(month_idx==mo)).cumsum().shift(1) for mo in range(1,13)}
            cum_cnt = {mo: (s.where(month_idx==mo).notna().astype(int)).cumsum().shift(1) for mo in range(1,13)}

            tms = test_months[(test_months['영업장명']==store) & (test_months['메뉴명']==menu)]['월_기간']
            for tm in tms:
                prev, prev2 = tm-1, tm-2
                m_prev  = float(s.get(prev, 0.0))
                m_prev2 = s.get(prev2, np.nan)
                cum_prev = float(cum.get(prev, 0.0))

                # 점유율/low-share
                l_share = (m_prev / cum_prev) if cum_prev > 0 else 0.0
                l_low   = int((cum_prev >= float(min_total_sales)) and (l_share <= float(share_threshold)))

                # 전월 변화
                if pd.isna(m_prev2) or (float(m_prev2) == 0.0):
                    l_abs = float(m_prev - (0.0 if pd.isna(m_prev2) else float(m_prev2)))
                    l_pct = 0.0
                else:
                    l_abs = float(m_prev - float(m_prev2))
                    l_pct = float(l_abs / float(m_prev2))

                # 급락
                l_drop = int((not pd.isna(m_prev2)) and (float(m_prev2) >= float(min_prev)) and
                             (l_pct <= float(sharp_drop_pct)) and (abs(l_abs) >= float(min_abs_drop)))

                # 좋은 달 분위
                past = s_shift.loc[:prev].dropna()
                if len(past) >= max(3, rolling_q_window//2):
                    thr = past.tail(rolling_q_window).quantile(good_month_quantile)
                    l_good = int(m_prev >= float(thr))
                else:
                    l_good = 0

                # MOY 상/하위
                mo_prev = prev.month
                vals, idxs = [], []
                for mo in range(1,13):
                    cs, cc = cum_sum[mo], cum_cnt[mo]
                    if prev in cs.index:
                        ssum, scnt = cs.loc[prev], cc.loc[prev]
                        if pd.notna(scnt) and scnt >= min_obs_moy and pd.notna(ssum):
                            vals.append(float(ssum/scnt)); idxs.append(mo-1)
                if len(vals) > 0:
                    order = np.argsort(vals)
                    top_idx = np.array(idxs)[order[-min(top_k,len(vals)):]] if top_k>0 else np.array([],dtype=int)
                    low_idx = np.array(idxs)[order[:min(bottom_k,len(vals))]] if bottom_k>0 else np.array([],dtype=int)
                    l_high = int((mo_prev-1) in top_idx)
                    l_lowm = int((mo_prev-1) in low_idx)
                else:
                    l_high = 0; l_lowm = 0

                feats_rows.append({
                    '영업장명': store, '메뉴명': menu, '월_기간': tm,
                    'lag1_month_share': np.float32(l_share),
                    'lag1_is_low_share_month': np.int8(l_low),
                    'lag1_monthly_sales': np.float32(m_prev),
                    'lag1_monthly_abs_change': np.float32(l_abs),
                    'lag1_monthly_pct_change': np.float32(l_pct),
                    'lag1_is_sharp_drop_month': np.int8(l_drop),
                    'lag1_is_good_month_quantile': np.int8(l_good),
                    'lag1_is_high_season_moy': np.int8(l_high),
                    'lag1_is_low_season_moy':  np.int8(l_lowm),
                })

        if feats_rows:
            lag_df = pd.DataFrame(feats_rows)
            df = df.merge(lag_df, on=['영업장명','메뉴명','월_기간'], how='left', suffixes=('', '_calc'))
            for col, dtype, fill in [
                ('lag1_month_share','float32',0.0),
                ('lag1_is_low_share_month','int8',0),
                ('lag1_monthly_sales','float32',0.0),
                ('lag1_monthly_abs_change','float32',0.0),
                ('lag1_monthly_pct_change','float32',0.0),
                ('lag1_is_good_month_quantile','int8',0),
                ('lag1_is_high_season_moy','int8',0),
            ]:
                src = col + '_calc'
                if src in df.columns:
                    df[col] = df[src]; df.drop(columns=[src], inplace=True)
                df[col] = df[col].fillna(fill).astype(dtype)

    # ── 4) 출시 이후 플래그 = is_probable_launch (요청대로) ─────────
    #     정의: 해당 (영업장명,메뉴명)의 first_sale_date 이후인 날짜면 1, 아니면 0
    if ('is_probable_launch' in exog_cols):
        if (first_sale_table is not None) and len(first_sale_table):
            fst = first_sale_table.copy()
            # 키 정렬
            if '영업장명' not in fst.columns or '메뉴명' not in fst.columns:
                if '영업장명_메뉴명' in fst.columns:
                    sp = fst['영업장명_메뉴명'].astype(str).str.split('_', n=1, expand=True)
                    fst['영업장명'] = sp[0]; fst['메뉴명'] = sp[1] if sp.shape[1]>1 else ''
                else:
                    raise KeyError("first_sale_table에는 ('영업장명','메뉴명') 또는 '영업장명_메뉴명'이 필요합니다.")
            if 'first_sale_date' not in fst.columns:
                raise KeyError("first_sale_table에는 'first_sale_date'가 필요합니다.")
            fst['first_sale_date'] = pd.to_datetime(fst['first_sale_date'])
            df = df.merge(fst[['영업장명','메뉴명','first_sale_date']], on=['영업장명','메뉴명'], how='left')
            df['is_probable_launch'] = (df['영업일자'] >= df['first_sale_date']).astype('int8').fillna(0)
            df.drop(columns=['first_sale_date'], inplace=True)
        else:
            # 정보 없으면 0으로
            if 'is_probable_launch' not in df.columns:
                df['is_probable_launch'] = 0

    # ── 5) 휴무일: 매장별 '연속 0매출(>=4일)' 구간의 월-일 합집합으로 설정 ─
    #     (훈련에서 만든 off_union_mday_df 전달 권장)
    if ('휴무일' in exog_cols):
        # 시작값(없으면 0)
        if '휴무일' not in df.columns:
            df['휴무일'] = 0
        if (off_union_mday_df is not None) and len(off_union_mday_df):
            offu = off_union_mday_df.copy()
            # 필요한 컬럼 체크
            for c in ['영업장명','month','day','is_union_offday']:
                if c not in offu.columns:
                    raise KeyError("off_union_mday_df에는 ['영업장명','month','day','is_union_offday']가 필요합니다.")
            tmp = df[['영업장명','month','day']].merge(
                offu, on=['영업장명','month','day'], how='left'
            )['is_union_offday'].fillna(0).astype('int8')
            # 기존 휴무일과 OR
            df['휴무일'] = ((df['휴무일'].fillna(0).astype(int) > 0) | (tmp == 1)).astype('int8')

    # ── 6) custom_offday: 휴무일==0에서만 규칙 적용 ────────────────
    if ('custom_offday' in exog_cols):
        if 'custom_offday' not in df.columns:
            df['custom_offday'] = 0
        # 월, 요일 준비 (숫자형)
        mo = df['month'].astype(int)
        wd = df['weekday'].astype(int)
        is_op = (df['휴무일'].fillna(0).astype(int) == 0).values  # 영업 중일 때만
        # 포레스트릿: 4,5,9,10,11 & 월요일
        mask = (
            (df['영업장명'] == '포레스트릿') &
            (mo.isin([4,5,9,10,11])) &
            (wd == 0) & is_op
        )
        df.loc[mask, 'custom_offday'] = 1
        # 연회장: 일요일
        mask = ((df['영업장명'] == '연회장') & (wd == 6) & is_op)
        df.loc[mask, 'custom_offday'] = 1
        # 화담숲주막/화담숲카페/라그로타/느티나무 셀프BBQ: 월요일
        mask = (
            df['영업장명'].isin(['화담숲주막','화담숲카페','라그로타','느티나무 셀프BBQ']) &
            (wd == 0) & is_op
        )
        df.loc[mask, 'custom_offday'] = 1
        df['custom_offday'] = df['custom_offday'].astype('int8')

    # ── 7) main_menu: 훈련에서의 집합 그대로 적용 ────────────────
    if ('is_main_menu' in exog_cols):
        if (main_menu_df is not None) and len(main_menu_df):
            mm = main_menu_df.copy()
            if '영업장명_메뉴명' in mm.columns and (('영업장명' not in mm.columns) or ('메뉴명' not in mm.columns)):
                # 그대로 merge
                if '영업장명_메뉴명' not in df.columns:
                    df['영업장명_메뉴명'] = df['영업장명'].astype(str) + '_' + df['메뉴명'].astype(str)
                df = df.merge(mm[['영업장명_메뉴명','is_main_menu']], on='영업장명_메뉴명', how='left')
            else:
                df = df.merge(mm[['영업장명','메뉴명','is_main_menu']], on=['영업장명','메뉴명'], how='left')
            df['is_main_menu'] = df['is_main_menu'].fillna(0).astype('int8')
        else:
            if 'is_main_menu' not in df.columns:
                df['is_main_menu'] = 0

    # ── 8) 기타 외생변수 기본값 ───────────────────────────────────
    default_zero = [
        'is_zero_sales_period', 'is_coldstart_60d'  # 필요시 0
    ]
    for c in default_zero:
        if c in exog_cols and c not in df.columns:
            df[c] = 0

    # object → 숫자
    for c in exog_cols:
        if c in df.columns and df[c].dtype == 'object':
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

    # 최종: exog_cols 모두 보유
    for c in exog_cols:
        if c not in df.columns:
            df[c] = 0

    return df


In [ ]:
@torch.no_grad()
def predict_next_7days_from_test(
    test_df: pd.DataFrame,
    trained_models: dict,
    *,
    # ✅ 훈련 구간에서 만든 (영업장명, 메뉴명, 월_기간, monthly_sales) 테이블 권장
    monthly_history: pd.DataFrame | None = None,
    # ✅ 전처리에 쓰일 매핑(훈련 데이터로 만든 표 전달)
    first_sale_table: pd.DataFrame | None = None,   # ['영업장명','메뉴명','first_sale_date'] 또는 '영업장명_메뉴명'
    main_menu_df: pd.DataFrame | None = None,       # ['영업장명','메뉴명','is_main_menu'] 또는 '영업장명_메뉴명'
    off_union_mday_df: pd.DataFrame | None = None,  # ['영업장명','month','day','is_union_offday']

    # 아래 파라미터는 훈련 때와 동일값 유지 권장 (lag1 월 피처 계산용)
    share_threshold: float = 0.05,
    min_total_sales: float | int = 1,
    sharp_drop_pct: float = -0.40,
    min_abs_drop: float = 20,
    min_prev: float = 30,
    good_month_quantile: float = 0.75,
    rolling_q_window: int = 12,
    top_k: int = 3,
    bottom_k: int = 3,
):
    """
    trained_models: train_lstm_per_store(df1) 결과(dict: store -> pack)
      - pack: {'model','menu2id','scalers','exog_cols', ...}
    test_df: 이 파일(최근 28일치)만 사용해 7일 예측
    monthly_history: (선택) 훈련 구간의 월별 집계. 있으면 lag1 피처가 정확해집니다.
    first_sale_table: 출시일 테이블(출시 이후 is_probable_launch=1 적용)
    main_menu_df:     훈련에서 산정한 주요메뉴 집합 그대로 반영
    off_union_mday_df:훈련에서 찾은 '연속 0매출(>=4일)' 구간의 (월,일) 합집합으로 휴무일=1 적용
    반환: ['영업장명','메뉴명','예측일자','예측수량']
    """
    # 0) 모든 매장에서 사용된 exog들의 합집합 (🔒누수 가능 원천은 차단)
    leaky_ban = {
        'is_high_season_moy','is_low_season_moy',
        'is_good_month_quantile','is_sharp_drop_month',
        'is_low_share_month','month_share',
        'monthly_sales','monthly_abs_change','monthly_pct_change',
    }
    exog_cols_all = sorted({
        c for pack in trained_models.values() for c in pack['exog_cols']
        if c not in leaky_ban
    })

    # 1) 테스트 전처리(✅ 학습과 동일 규칙, lag1 피처 계산 + 출시/휴무/커스텀오프/주요메뉴 반영)
    df = preprocess_test_df_for_inference(
        test_df,
        exog_cols=exog_cols_all,
        monthly_history=monthly_history,
        first_sale_table=first_sale_table,
        main_menu_df=main_menu_df,
        off_union_mday_df=off_union_mday_df,
        # lag1 계산 하이퍼파라미터(훈련값과 일치 권장)
        share_threshold=share_threshold,
        min_total_sales=min_total_sales,
        sharp_drop_pct=sharp_drop_pct,
        min_abs_drop=min_abs_drop,
        min_prev=min_prev,
        good_month_quantile=good_month_quantile,
        rolling_q_window=rolling_q_window,
        top_k=top_k,
        bottom_k=bottom_k,
    )

    preds = []
    for store, pack in trained_models.items():
        sdf = df[df['영업장명'] == store].copy()
        if sdf.empty:
            continue

        last_date = sdf['영업일자'].max()
        menu2id  = pack['menu2id']
        scalers  = pack['scalers']
        # 매장별 exog도 안전하게 누수 컬럼 제거 + 실제 보유 컬럼만 사용
        exog_cols = [c for c in pack['exog_cols'] if (c in sdf.columns) and (c not in leaky_ban)]
        model    = pack['model']
        model.eval()

        for menu, g in sdf.groupby('메뉴명'):
            if (menu not in menu2id) or (menu not in scalers):
                # 학습 제외 메뉴일 수 있음
                continue
            g = g.sort_values('영업일자').copy()
            if len(g) < LOOKBACK:
                continue

            # 스케일 & 피처 구성
            scaler = scalers[menu]
            g['scaled_sales'] = scaler.transform(g[['매출수량']])
            feats = ['scaled_sales'] + exog_cols
            arr = g[feats].fillna(0).values.astype('float32')

            X    = torch.tensor(arr[-LOOKBACK:], dtype=torch.float32).unsqueeze(0).to(DEVICE)
            m_id = torch.tensor([menu2id[menu]], dtype=torch.long).to(DEVICE)

            yhat_scaled = model(X, m_id).cpu().numpy().ravel()
            yhat = scaler.inverse_transform(yhat_scaled.reshape(-1, 1)).ravel()

            future_days = pd.date_range(last_date + pd.Timedelta(days=1), periods=HORIZON, freq='D')
            for d, val in zip(future_days, yhat):
                preds.append([store, menu, d, float(max(val, 0.0))])

    pred_df = pd.DataFrame(preds, columns=['영업장명','메뉴명','예측일자','예측수량'])
    return pred_df


In [58]:
all_preds = []
for path in sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/LGaimers/해커톤/test/TEST_*.csv')):
    test_df = pd.read_csv(path)
    pred_df = predict_next_7days_from_test(
        test_df,
        trained_models,
        monthly_history=monthly_history,     # 권장
        first_sale_table=first_sale_table,   # ✅ 추가
        main_menu_df=main_menu_df,           # ✅ 추가
        off_union_mday_df=off_union_mday_df, # ✅ 추가
        share_threshold=0.05, min_total_sales=1,
        sharp_drop_pct=-0.40, min_abs_drop=20, min_prev=30,
        good_month_quantile=0.75, rolling_q_window=12,
        top_k=3, bottom_k=3
    )
    pred_df['source'] = os.path.basename(path)  # (선택) 추적용
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)
print(full_pred_df.head())

         영업장명      메뉴명       예측일자      예측수량       source
0  느티나무 셀프BBQ  1인 수저세트 2024-07-14  5.726124  TEST_00.csv
1  느티나무 셀프BBQ  1인 수저세트 2024-07-15  4.982970  TEST_00.csv
2  느티나무 셀프BBQ  1인 수저세트 2024-07-16  2.672959  TEST_00.csv
3  느티나무 셀프BBQ  1인 수저세트 2024-07-17  3.315212  TEST_00.csv
4  느티나무 셀프BBQ  1인 수저세트 2024-07-18  3.541336  TEST_00.csv


In [ ]:
import pandas as pd
import numpy as np
import re

def convert_predictions_to_submission_flexible(
    pred_df: pd.DataFrame,
    template_csv_path: str,
    output_csv_path: str,
    round_to_int: bool = True,
    horizon: int = 7
) -> pd.DataFrame:
    sub = pd.read_csv(template_csv_path, dtype=str).copy()
    if '영업일자' not in sub.columns:
        raise ValueError("템플릿에 '영업일자' 열이 없습니다.")
    target_cols = [c for c in sub.columns if c != '영업일자']
    id_values = sub['영업일자'].astype(str)
    id_mode = id_values.str.match(r'^TEST_\d+\+\d+일$').all()

    pred = pred_df.copy()
    if '영업장명_메뉴명' not in pred.columns:
        pred['영업장명_메뉴명'] = pred['영업장명'].astype(str) + '_' + pred['메뉴명'].astype(str)

    if id_mode:
        if 'source' not in pred.columns:
            raise ValueError("ID 모드 변환에는 pred_df에 'source' 컬럼이 필요합니다.")

        def _prefix_from_source(s):
            m = re.search(r'(TEST_\d+)', str(s))
            return m.group(1) if m else None

        pred['prefix'] = pred['source'].map(_prefix_from_source)
        pred = pred.sort_values(['prefix','영업장명_메뉴명','예측일자'])
        pred['일오프셋'] = pred.groupby(['prefix','영업장명_메뉴명']).cumcount() + 1
        pred = pred[pred['일오프셋'].between(1, horizon)]
        pred['row_id'] = pred['prefix'] + '+' + pred['일오프셋'].astype(str) + '일'

        pivot = pred.pivot_table(index='row_id',
                                 columns='영업장명_메뉴명',
                                 values='예측수량',
                                 aggfunc='sum').reindex(columns=target_cols)

        # 🔧 핵심: 키로 병합
        out = sub[['영업일자']].merge(pivot, left_on='영업일자', right_index=True, how='left')

    else:
        # 날짜 템플릿
        sub['_dt'] = pd.to_datetime(sub['영업일자'], errors='coerce', format='%Y-%m-%d')
        if sub['_dt'].isna().any():
            sub['_dt'] = pd.to_datetime(sub['영업일자'], errors='coerce')
        if sub['_dt'].isna().any():
            bad = sub['영업일자'][sub['_dt'].isna()].unique()[:5]
            raise ValueError(f"템플릿 날짜 파싱 실패. 예: {bad}")

        pred['예측일자'] = pd.to_datetime(pred['예측일자'])
        pivot = pred.pivot_table(index='예측일자',
                                 columns='영업장명_메뉴명',
                                 values='예측수량',
                                 aggfunc='sum').reindex(columns=target_cols)

        # 🔧 핵심: 날짜 키로 병합
        out = sub[['영업일자','_dt']].merge(pivot, left_on='_dt', right_index=True, how='left').drop(columns=['_dt'])

    # 후처리
    for c in target_cols:
        out[c] = pd.to_numeric(out[c], errors='coerce').fillna(0)
    out[target_cols] = out[target_cols].clip(lower=0)
    if round_to_int:
        out[target_cols] = out[target_cols].round().astype(int)

    # 최종 열 순서 정리
    out = out[['영업일자'] + target_cols]
    out.to_csv(output_csv_path, index=False)
    print(f"✅ 저장 완료: {output_csv_path}")
    return out


In [ ]:
converted = convert_predictions_to_submission_flexible(
    pred_df=full_pred_df,
    template_csv_path='/content/drive/MyDrive/Colab Notebooks/LGaimers/해커톤/sample_submission.csv',
    output_csv_path='/content/drive/MyDrive/Colab Notebooks/LGaimers/해커톤/train/submission_1.csv',
    round_to_int=True
)
converted.to_csv(
    '/content/drive/MyDrive/Colab Notebooks/LGaimers/해커톤/train/submission_1_utf8sig.csv',
    index=False, encoding='utf-8-sig'
)
